In [1]:
# Import libraries

from library.common import Database
import pathlib

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
# import seaborn as sns 
# import altair as alt
import warnings
warnings.filterwarnings("ignore")
# from vega_datasets import data
# from sklearn.linear_model import LinearRegression
# import statsmodels.api as sm

# Display all columns of pandas dataframes
pd.set_option('display.max_columns', None)

db = Database()
write_to_db = True

# Get the path to raw data files
# path = pathlib.Path.cwd().parent.joinpath(path_to_data)

## Data sources

### (A) Carbon dioxide emission (production-based concept)

Production-based carbon dioxide emissions refer to emissions from all the oil, coal, and gas consumed in a country by private households, industrial production of goods and services, and electricity production.  While some of the goods and services produced by country A will be exported to country B for consumption, the emission from relevant production process is counted in country A, not country B.

Originally, we incline to use the dataset obtained from a paper 'Global Carbon Budget 2020' by Friedlingstein et al. (2020), which have emission data from 1960 to 2019 (https://essd.copernicus.org/articles/12/3269/2020/)

Later, we note that CO2 emission data in 'Our World in Data' (https://ourworldindata.org/co2-emissions) also sourced from 'Global Carbon Budget' and includes the latest update (i.e., 2020 emission data) released on Nov 5, 2021.  Therefore, we finally decide to use the carbon dioxide emission data downloaded from 'Our World in Data' website.

CO2 emission is recorded in million ton in the data source, and was converted to thousand ton for data analysis purpose.

Note:
There is another source of carbon dioxide emission from World Bank (https://data.worldbank.org/indicator/EN.ATM.CO2E.KT). The figures from World Bank were not used because World Bank only have figures up to 2018.

In [2]:
# Read the tables
# Get the path to raw data files
path_to_data = r'data/raw/manifest'
path = str(pathlib.Path.cwd().parent.joinpath(path_to_data))+r"/"
#
# # CO2 emission dataset from 'Our World in Data' website, which is sourced from Global Carbon Project 2021 (released on Nov 5,2021)
# co2_emission_owid = pd.read_csv(path+'owid-co2-data.csv', sep=",")

# path_to_dataset = \
#     r'https://github.com/owid/co2-data/raw/master/owid-co2-data.xlsx'
# name_of_dataset = "owid_CO2_Greenhouse_Gas_Emissions"
# co2_emission_owid = pd.read_excel(path_to_dataset)

table = 'owid_co2_greenhouse_gas_emissions'
co2_emission_owid = pd.read_sql(table, db.config)
co2_emission = co2_emission_owid[['iso_code','year','co2']]
co2_emission = co2_emission[co2_emission['year']>=1960]
co2_emission.rename(columns={'year':'Year','iso_code':'Country Code','co2':'CO2 Emission'}, inplace=True)
co2_emission['CO2 Emission']=co2_emission['CO2 Emission']*1000 #convert unit from million ton to thousand ton

co2_world_emission = co2_emission[co2_emission['Country Code']=='OWID_WRL']
co2_world_emission = co2_world_emission[['CO2 Emission']].reset_index(drop=True)

co2_emission = co2_emission[co2_emission['Country Code'].notna()]
not_use=['OWID_WRL']
co2_emission = co2_emission[~co2_emission['Country Code'].isin(not_use)]

Carbon dioxide emission by sector from 1990 to 2018 are available from Climate Watch (https://www.climatewatchdata.org/ghg-emissions?end_year=2018&gases=co2&sectors=energy&start_year=1990).  The sectoral breakdowns follow the definition adopted by the Intergovernmental Panel on Climate Change (IPCC):
* Energy (with further breakdowns to electricity and heat; manufacturing and construction; transport; other fuel combustion and fugitive emissions)
* Industrial processes (e.g. cement, aluminium, nitric acid production)
* Agriculture
* Waste (e.g. landfill, waste water treatment)
* International bunkers (i.e., aviation and marine bunkers)

Units of CO2 emission by sector are recorded in million ton, and were converted to thousand ton for analysis purpose.

In [3]:
# Datasets from Climate Watch (CO2 emission by usage)
# co2_electricity = pd.read_csv(path+'co2_energy_electricity and heat.csv') # CO2 emission from electricity and heat generation, in million tonne (Mt)
# co2_building = pd.read_csv(path+'co2_energy_building.csv') # CO2 emission from energy use in building, in million tonne (Mt)
# co2_manufacturing = pd.read_csv(path+'co2_energy_manufacturing.csv') # CO2 emission from energy use in manufacturing, in million tonne (Mt)
# co2_transport = pd.read_csv(path+'co2_energy_transportation.csv') # CO2 emission from energy use in transportation, in million tonne (Mt)
# co2_other_fuel = pd.read_csv(path+'co2_energy_other fuel consumption.csv') # CO2 emission from energy use in other fuel consumption, in million tonne (Mt)
# co2_fugitive = pd.read_csv(path+'co2_energy_fugitive emission.csv') # CO2 emission from fugitive emission in energy use, in million tonne (Mt)
# co2_energy_total = pd.read_csv(path+'co2_energy_total.csv') # CO2 emission from energy use in transportation, in million tonne (Mt)
# co2_bunkers = pd.read_csv(path+'co2_bunker fuel.csv') # CO2 emission from bunkers fuel, in million tonne
# co2_industrial_process = pd.read_csv(path+'co2_industrial process.csv') # CO2 emission from industrial process, in million tonne

# Datasets from Climate Watch (CO2 emission by usage)
co2_electricity = pd.read_sql('co2_energy_electricity_and_heat', db.config) # CO2 emission from electricity and heat generation, in million tonne (Mt)
co2_building = pd.read_sql('co2_energy_building', db.config) # CO2 emission from energy use in building, in million tonne (Mt)
co2_manufacturing = pd.read_sql('co2_energy_manufacturing', db.config) # CO2 emission from energy use in manufacturing, in million tonne (Mt)
co2_transport = pd.read_sql('co2_energy_transportation', db.config) # CO2 emission from energy use in transportation, in million tonne (Mt)
co2_other_fuel = pd.read_sql('co2_energy_other_fuel_consumption', db.config) # CO2 emission from energy use in other fuel consumption, in million tonne (Mt)
co2_fugitive = pd.read_sql('co2_energy_fugitive_emission', db.config) # CO2 emission from fugitive emission in energy use, in million tonne (Mt)
co2_energy_total = pd.read_sql('co2_energy_total', db.config) # CO2 emission from energy use in transportation, in million tonne (Mt)
co2_bunkers = pd.read_sql('co2_bunker_fuel', db.config) # CO2 emission from bunkers fuel, in million tonne
co2_industrial_process = pd.read_sql('co2_industrial_process', db.config) # CO2 emission from industrial process, in million tonne


In [4]:
# Rename the country name used in Climate Watch to the name used in World Bank
co2_electricity.loc[(co2_electricity['Country/Region'] == 'Bahamas'),'Country/Region']='Bahamas, The'
co2_electricity.loc[(co2_electricity['Country/Region'] == 'Brunei'),'Country/Region']='Brunei Darussalam'
co2_electricity.loc[(co2_electricity['Country/Region'] == 'Cape Verde'),'Country/Region']='Cabo Verde'
co2_electricity.loc[(co2_electricity['Country/Region'] == "CÃ´te d'Ivoire"),'Country/Region']="Cote d'Ivoire"
co2_electricity.loc[(co2_electricity['Country/Region'] == 'Democratic Republic of the Congo'),'Country/Region']='Congo, Dem. Rep.'
co2_electricity.loc[(co2_electricity['Country/Region'] == 'Egypt'),'Country/Region']='Egypt, Arab Rep.'
co2_electricity.loc[(co2_electricity['Country/Region'] == 'Gambia'),'Country/Region']='Gambia, The'
co2_electricity.loc[(co2_electricity['Country/Region'] == 'Iran'),'Country/Region']='Iran, Islamic Rep.'
co2_electricity.loc[(co2_electricity['Country/Region'] == 'Kyrgyzstan'),'Country/Region']='Kyrgyz Republic'
co2_electricity.loc[(co2_electricity['Country/Region'] == 'Laos'),'Country/Region']='Lao PDR'
co2_electricity.loc[(co2_electricity['Country/Region'] == 'Micronesia'),'Country/Region']='Micronesia, Fed. Sts.'
co2_electricity.loc[(co2_electricity['Country/Region'] == 'Macedonia'),'Country/Region']='North Macedonia'
co2_electricity.loc[(co2_electricity['Country/Region'] == 'North Korea'),'Country/Region']="Korea, Dem. People's Rep."
co2_electricity.loc[(co2_electricity['Country/Region'] == 'Republic of Congo'),'Country/Region']='Congo, Rep.'
co2_electricity.loc[(co2_electricity['Country/Region'] == 'Russia'),'Country/Region']='Russian Federation'
co2_electricity.loc[(co2_electricity['Country/Region'] == 'Saint Kitts and Nevis'),'Country/Region']='St. Kitts and Nevis'
co2_electricity.loc[(co2_electricity['Country/Region'] == 'Saint Vincent and the Grenadines'),'Country/Region']='St. Vincent and the Grenadines'
co2_electricity.loc[(co2_electricity['Country/Region'] == 'Slovakia'),'Country/Region']='Slovak Republic'
co2_electricity.loc[(co2_electricity['Country/Region'] == 'South Korea'),'Country/Region']='Korea, Rep.'
co2_electricity.loc[(co2_electricity['Country/Region'] == 'Syria'),'Country/Region']='Syrian Arab Republic'
co2_electricity.loc[(co2_electricity['Country/Region'] == 'Venezuela'),'Country/Region']='Venezuela, RB'
co2_electricity.loc[(co2_electricity['Country/Region'] == 'Yemen'),'Country/Region']='Yemen, Rep.'

In [5]:
co2_building.loc[(co2_building['Country/Region'] == 'Bahamas'),'Country/Region']='Bahamas, The'
co2_building.loc[(co2_building['Country/Region'] == 'Brunei'),'Country/Region']='Brunei Darussalam'
co2_building.loc[(co2_building['Country/Region'] == 'Cape Verde'),'Country/Region']='Cabo Verde'
co2_building.loc[(co2_building['Country/Region'] == "CÃ´te d'Ivoire"),'Country/Region']="Cote d'Ivoire"
co2_building.loc[(co2_building['Country/Region'] == 'Democratic Republic of the Congo'),'Country/Region']='Congo, Dem. Rep.'
co2_building.loc[(co2_building['Country/Region'] == 'Egypt'),'Country/Region']='Egypt, Arab Rep.'
co2_building.loc[(co2_building['Country/Region'] == 'Gambia'),'Country/Region']='Gambia, The'
co2_building.loc[(co2_building['Country/Region'] == 'Iran'),'Country/Region']='Iran, Islamic Rep.'
co2_building.loc[(co2_building['Country/Region'] == 'Kyrgyzstan'),'Country/Region']='Kyrgyz Republic'
co2_building.loc[(co2_building['Country/Region'] == 'Laos'),'Country/Region']='Lao PDR'
co2_building.loc[(co2_building['Country/Region'] == 'Micronesia'),'Country/Region']='Micronesia, Fed. Sts.'
co2_building.loc[(co2_building['Country/Region'] == 'Macedonia'),'Country/Region']='North Macedonia'
co2_building.loc[(co2_building['Country/Region'] == 'North Korea'),'Country/Region']="Korea, Dem. People's Rep."
co2_building.loc[(co2_building['Country/Region'] == 'Republic of Congo'),'Country/Region']='Congo, Rep.'
co2_building.loc[(co2_building['Country/Region'] == 'Russia'),'Country/Region']='Russian Federation'
co2_building.loc[(co2_building['Country/Region'] == 'Saint Kitts and Nevis'),'Country/Region']='St. Kitts and Nevis'
co2_building.loc[(co2_building['Country/Region'] == 'Saint Vincent and the Grenadines'),'Country/Region']='St. Vincent and the Grenadines'
co2_building.loc[(co2_building['Country/Region'] == 'Slovakia'),'Country/Region']='Slovak Republic'
co2_building.loc[(co2_building['Country/Region'] == 'South Korea'),'Country/Region']='Korea, Rep.'
co2_building.loc[(co2_building['Country/Region'] == 'Syria'),'Country/Region']='Syrian Arab Republic'
co2_building.loc[(co2_building['Country/Region'] == 'Venezuela'),'Country/Region']='Venezuela, RB'
co2_building.loc[(co2_building['Country/Region'] == 'Yemen'),'Country/Region']='Yemen, Rep.'

In [6]:
co2_manufacturing.loc[(co2_manufacturing['Country/Region'] == 'Bahamas'),'Country/Region']='Bahamas, The'
co2_manufacturing.loc[(co2_manufacturing['Country/Region'] == 'Brunei'),'Country/Region']='Brunei Darussalam'
co2_manufacturing.loc[(co2_manufacturing['Country/Region'] == 'Cape Verde'),'Country/Region']='Cabo Verde'
co2_manufacturing.loc[(co2_manufacturing['Country/Region'] == "CÃ´te d'Ivoire"),'Country/Region']="Cote d'Ivoire"
co2_manufacturing.loc[(co2_manufacturing['Country/Region'] == 'Democratic Republic of the Congo'),'Country/Region']='Congo, Dem. Rep.'
co2_manufacturing.loc[(co2_manufacturing['Country/Region'] == 'Egypt'),'Country/Region']='Egypt, Arab Rep.'
co2_manufacturing.loc[(co2_manufacturing['Country/Region'] == 'Gambia'),'Country/Region']='Gambia, The'
co2_manufacturing.loc[(co2_manufacturing['Country/Region'] == 'Iran'),'Country/Region']='Iran, Islamic Rep.'
co2_manufacturing.loc[(co2_manufacturing['Country/Region'] == 'Kyrgyzstan'),'Country/Region']='Kyrgyz Republic'
co2_manufacturing.loc[(co2_manufacturing['Country/Region'] == 'Laos'),'Country/Region']='Lao PDR'
co2_manufacturing.loc[(co2_manufacturing['Country/Region'] == 'Micronesia'),'Country/Region']='Micronesia, Fed. Sts.'
co2_manufacturing.loc[(co2_manufacturing['Country/Region'] == 'Macedonia'),'Country/Region']='North Macedonia'
co2_manufacturing.loc[(co2_manufacturing['Country/Region'] == 'North Korea'),'Country/Region']="Korea, Dem. People's Rep."
co2_manufacturing.loc[(co2_manufacturing['Country/Region'] == 'Republic of Congo'),'Country/Region']='Congo, Rep.'
co2_manufacturing.loc[(co2_manufacturing['Country/Region'] == 'Russia'),'Country/Region']='Russian Federation'
co2_manufacturing.loc[(co2_manufacturing['Country/Region'] == 'Saint Kitts and Nevis'),'Country/Region']='St. Kitts and Nevis'
co2_manufacturing.loc[(co2_manufacturing['Country/Region'] == 'Saint Vincent and the Grenadines'),'Country/Region']='St. Vincent and the Grenadines'
co2_manufacturing.loc[(co2_manufacturing['Country/Region'] == 'Slovakia'),'Country/Region']='Slovak Republic'
co2_manufacturing.loc[(co2_manufacturing['Country/Region'] == 'South Korea'),'Country/Region']='Korea, Rep.'
co2_manufacturing.loc[(co2_manufacturing['Country/Region'] == 'Syria'),'Country/Region']='Syrian Arab Republic'
co2_manufacturing.loc[(co2_manufacturing['Country/Region'] == 'Venezuela'),'Country/Region']='Venezuela, RB'
co2_manufacturing.loc[(co2_manufacturing['Country/Region'] == 'Yemen'),'Country/Region']='Yemen, Rep.'

In [7]:
co2_other_fuel.loc[(co2_other_fuel['Country/Region'] == 'Bahamas'),'Country/Region']='Bahamas, The'
co2_other_fuel.loc[(co2_other_fuel['Country/Region'] == 'Brunei'),'Country/Region']='Brunei Darussalam'
co2_other_fuel.loc[(co2_other_fuel['Country/Region'] == 'Cape Verde'),'Country/Region']='Cabo Verde'
co2_other_fuel.loc[(co2_other_fuel['Country/Region'] == "CÃ´te d'Ivoire"),'Country/Region']="Cote d'Ivoire"
co2_other_fuel.loc[(co2_other_fuel['Country/Region'] == 'Democratic Republic of the Congo'),'Country/Region']='Congo, Dem. Rep.'
co2_other_fuel.loc[(co2_other_fuel['Country/Region'] == 'Egypt'),'Country/Region']='Egypt, Arab Rep.'
co2_other_fuel.loc[(co2_other_fuel['Country/Region'] == 'Gambia'),'Country/Region']='Gambia, The'
co2_other_fuel.loc[(co2_other_fuel['Country/Region'] == 'Iran'),'Country/Region']='Iran, Islamic Rep.'
co2_other_fuel.loc[(co2_other_fuel['Country/Region'] == 'Kyrgyzstan'),'Country/Region']='Kyrgyz Republic'
co2_other_fuel.loc[(co2_other_fuel['Country/Region'] == 'Laos'),'Country/Region']='Lao PDR'
co2_other_fuel.loc[(co2_other_fuel['Country/Region'] == 'Micronesia'),'Country/Region']='Micronesia, Fed. Sts.'
co2_other_fuel.loc[(co2_other_fuel['Country/Region'] == 'Macedonia'),'Country/Region']='North Macedonia'
co2_other_fuel.loc[(co2_other_fuel['Country/Region'] == 'North Korea'),'Country/Region']="Korea, Dem. People's Rep."
co2_other_fuel.loc[(co2_other_fuel['Country/Region'] == 'Republic of Congo'),'Country/Region']='Congo, Rep.'
co2_other_fuel.loc[(co2_other_fuel['Country/Region'] == 'Russia'),'Country/Region']='Russian Federation'
co2_other_fuel.loc[(co2_other_fuel['Country/Region'] == 'Saint Kitts and Nevis'),'Country/Region']='St. Kitts and Nevis'
co2_other_fuel.loc[(co2_other_fuel['Country/Region'] == 'Saint Vincent and the Grenadines'),'Country/Region']='St. Vincent and the Grenadines'
co2_other_fuel.loc[(co2_other_fuel['Country/Region'] == 'Slovakia'),'Country/Region']='Slovak Republic'
co2_other_fuel.loc[(co2_other_fuel['Country/Region'] == 'South Korea'),'Country/Region']='Korea, Rep.'
co2_other_fuel.loc[(co2_other_fuel['Country/Region'] == 'Syria'),'Country/Region']='Syrian Arab Republic'
co2_other_fuel.loc[(co2_other_fuel['Country/Region'] == 'Venezuela'),'Country/Region']='Venezuela, RB'
co2_other_fuel.loc[(co2_other_fuel['Country/Region'] == 'Yemen'),'Country/Region']='Yemen, Rep.'

In [8]:
co2_fugitive.loc[(co2_fugitive['Country/Region'] == 'Bahamas'),'Country/Region']='Bahamas, The'
co2_fugitive.loc[(co2_fugitive['Country/Region'] == 'Brunei'),'Country/Region']='Brunei Darussalam'
co2_fugitive.loc[(co2_fugitive['Country/Region'] == 'Cape Verde'),'Country/Region']='Cabo Verde'
co2_fugitive.loc[(co2_fugitive['Country/Region'] == "CÃ´te d'Ivoire"),'Country/Region']="Cote d'Ivoire"
co2_fugitive.loc[(co2_fugitive['Country/Region'] == 'Democratic Republic of the Congo'),'Country/Region']='Congo, Dem. Rep.'
co2_fugitive.loc[(co2_fugitive['Country/Region'] == 'Egypt'),'Country/Region']='Egypt, Arab Rep.'
co2_fugitive.loc[(co2_fugitive['Country/Region'] == 'Gambia'),'Country/Region']='Gambia, The'
co2_fugitive.loc[(co2_fugitive['Country/Region'] == 'Iran'),'Country/Region']='Iran, Islamic Rep.'
co2_fugitive.loc[(co2_fugitive['Country/Region'] == 'Kyrgyzstan'),'Country/Region']='Kyrgyz Republic'
co2_fugitive.loc[(co2_fugitive['Country/Region'] == 'Laos'),'Country/Region']='Lao PDR'
co2_fugitive.loc[(co2_fugitive['Country/Region'] == 'Micronesia'),'Country/Region']='Micronesia, Fed. Sts.'
co2_fugitive.loc[(co2_fugitive['Country/Region'] == 'Macedonia'),'Country/Region']='North Macedonia'
co2_fugitive.loc[(co2_fugitive['Country/Region'] == 'North Korea'),'Country/Region']="Korea, Dem. People's Rep."
co2_fugitive.loc[(co2_fugitive['Country/Region'] == 'Republic of Congo'),'Country/Region']='Congo, Rep.'
co2_fugitive.loc[(co2_fugitive['Country/Region'] == 'Russia'),'Country/Region']='Russian Federation'
co2_fugitive.loc[(co2_fugitive['Country/Region'] == 'Saint Kitts and Nevis'),'Country/Region']='St. Kitts and Nevis'
co2_fugitive.loc[(co2_fugitive['Country/Region'] == 'Saint Vincent and the Grenadines'),'Country/Region']='St. Vincent and the Grenadines'
co2_fugitive.loc[(co2_fugitive['Country/Region'] == 'Slovakia'),'Country/Region']='Slovak Republic'
co2_fugitive.loc[(co2_fugitive['Country/Region'] == 'South Korea'),'Country/Region']='Korea, Rep.'
co2_fugitive.loc[(co2_fugitive['Country/Region'] == 'Syria'),'Country/Region']='Syrian Arab Republic'
co2_fugitive.loc[(co2_fugitive['Country/Region'] == 'Venezuela'),'Country/Region']='Venezuela, RB'
co2_fugitive.loc[(co2_fugitive['Country/Region'] == 'Yemen'),'Country/Region']='Yemen, Rep.'

In [9]:
co2_bunkers.loc[(co2_bunkers['Country/Region'] == 'Bahamas'),'Country/Region']='Bahamas, The'
co2_bunkers.loc[(co2_bunkers['Country/Region'] == 'Brunei'),'Country/Region']='Brunei Darussalam'
co2_bunkers.loc[(co2_bunkers['Country/Region'] == 'Cape Verde'),'Country/Region']='Cabo Verde'
co2_bunkers.loc[(co2_bunkers['Country/Region'] == "CÃ´te d'Ivoire"),'Country/Region']="Cote d'Ivoire"
co2_bunkers.loc[(co2_bunkers['Country/Region'] == 'Democratic Republic of the Congo'),'Country/Region']='Congo, Dem. Rep.'
co2_bunkers.loc[(co2_bunkers['Country/Region'] == 'Egypt'),'Country/Region']='Egypt, Arab Rep.'
co2_bunkers.loc[(co2_bunkers['Country/Region'] == 'Gambia'),'Country/Region']='Gambia, The'
co2_bunkers.loc[(co2_bunkers['Country/Region'] == 'Iran'),'Country/Region']='Iran, Islamic Rep.'
co2_bunkers.loc[(co2_bunkers['Country/Region'] == 'Kyrgyzstan'),'Country/Region']='Kyrgyz Republic'
co2_bunkers.loc[(co2_bunkers['Country/Region'] == 'Laos'),'Country/Region']='Lao PDR'
co2_bunkers.loc[(co2_bunkers['Country/Region'] == 'Micronesia'),'Country/Region']='Micronesia, Fed. Sts.'
co2_bunkers.loc[(co2_bunkers['Country/Region'] == 'Macedonia'),'Country/Region']='North Macedonia'
co2_bunkers.loc[(co2_bunkers['Country/Region'] == 'North Korea'),'Country/Region']="Korea, Dem. People's Rep."
co2_bunkers.loc[(co2_bunkers['Country/Region'] == 'Republic of Congo'),'Country/Region']='Congo, Rep.'
co2_bunkers.loc[(co2_bunkers['Country/Region'] == 'Russia'),'Country/Region']='Russian Federation'
co2_bunkers.loc[(co2_bunkers['Country/Region'] == 'Saint Kitts and Nevis'),'Country/Region']='St. Kitts and Nevis'
co2_bunkers.loc[(co2_bunkers['Country/Region'] == 'Saint Vincent and the Grenadines'),'Country/Region']='St. Vincent and the Grenadines'
co2_bunkers.loc[(co2_bunkers['Country/Region'] == 'Slovakia'),'Country/Region']='Slovak Republic'
co2_bunkers.loc[(co2_bunkers['Country/Region'] == 'South Korea'),'Country/Region']='Korea, Rep.'
co2_bunkers.loc[(co2_bunkers['Country/Region'] == 'Syria'),'Country/Region']='Syrian Arab Republic'
co2_bunkers.loc[(co2_bunkers['Country/Region'] == 'Venezuela'),'Country/Region']='Venezuela, RB'
co2_bunkers.loc[(co2_bunkers['Country/Region'] == 'Yemen'),'Country/Region']='Yemen, Rep.'

In [10]:
co2_industrial_process.loc[(co2_industrial_process['Country/Region'] == 'Bahamas'),'Country/Region']='Bahamas, The'
co2_industrial_process.loc[(co2_industrial_process['Country/Region'] == 'Brunei'),'Country/Region']='Brunei Darussalam'
co2_industrial_process.loc[(co2_industrial_process['Country/Region'] == 'Cape Verde'),'Country/Region']='Cabo Verde'
co2_industrial_process.loc[(co2_industrial_process['Country/Region'] == "CÃ´te d'Ivoire"),'Country/Region']="Cote d'Ivoire"
co2_industrial_process.loc[(co2_industrial_process['Country/Region'] == 'Democratic Republic of the Congo'),'Country/Region']='Congo, Dem. Rep.'
co2_industrial_process.loc[(co2_industrial_process['Country/Region'] == 'Egypt'),'Country/Region']='Egypt, Arab Rep.'
co2_industrial_process.loc[(co2_industrial_process['Country/Region'] == 'Gambia'),'Country/Region']='Gambia, The'
co2_industrial_process.loc[(co2_industrial_process['Country/Region'] == 'Iran'),'Country/Region']='Iran, Islamic Rep.'
co2_industrial_process.loc[(co2_industrial_process['Country/Region'] == 'Kyrgyzstan'),'Country/Region']='Kyrgyz Republic'
co2_industrial_process.loc[(co2_industrial_process['Country/Region'] == 'Laos'),'Country/Region']='Lao PDR'
co2_industrial_process.loc[(co2_industrial_process['Country/Region'] == 'Micronesia'),'Country/Region']='Micronesia, Fed. Sts.'
co2_industrial_process.loc[(co2_industrial_process['Country/Region'] == 'Macedonia'),'Country/Region']='North Macedonia'
co2_industrial_process.loc[(co2_industrial_process['Country/Region'] == 'North Korea'),'Country/Region']="Korea, Dem. People's Rep."
co2_industrial_process.loc[(co2_industrial_process['Country/Region'] == 'Republic of Congo'),'Country/Region']='Congo, Rep.'
co2_industrial_process.loc[(co2_industrial_process['Country/Region'] == 'Russia'),'Country/Region']='Russian Federation'
co2_industrial_process.loc[(co2_industrial_process['Country/Region'] == 'Saint Kitts and Nevis'),'Country/Region']='St. Kitts and Nevis'
co2_industrial_process.loc[(co2_industrial_process['Country/Region'] == 'Saint Vincent and the Grenadines'),'Country/Region']='St. Vincent and the Grenadines'
co2_industrial_process.loc[(co2_industrial_process['Country/Region'] == 'Slovakia'),'Country/Region']='Slovak Republic'
co2_industrial_process.loc[(co2_industrial_process['Country/Region'] == 'South Korea'),'Country/Region']='Korea, Rep.'
co2_industrial_process.loc[(co2_industrial_process['Country/Region'] == 'Syria'),'Country/Region']='Syrian Arab Republic'
co2_industrial_process.loc[(co2_industrial_process['Country/Region'] == 'Venezuela'),'Country/Region']='Venezuela, RB'
co2_industrial_process.loc[(co2_industrial_process['Country/Region'] == 'Yemen'),'Country/Region']='Yemen, Rep.'

In [11]:
co2_transport.loc[(co2_industrial_process['Country/Region'] == 'Bahamas'),'Country/Region']='Bahamas, The'
co2_transport.loc[(co2_industrial_process['Country/Region'] == 'Brunei'),'Country/Region']='Brunei Darussalam'
co2_transport.loc[(co2_industrial_process['Country/Region'] == 'Cape Verde'),'Country/Region']='Cabo Verde'
co2_transport.loc[(co2_industrial_process['Country/Region'] == "CÃ´te d'Ivoire"),'Country/Region']="Cote d'Ivoire"
co2_transport.loc[(co2_industrial_process['Country/Region'] == 'Democratic Republic of the Congo'),'Country/Region']='Congo, Dem. Rep.'
co2_transport.loc[(co2_industrial_process['Country/Region'] == 'Egypt'),'Country/Region']='Egypt, Arab Rep.'
co2_transport.loc[(co2_industrial_process['Country/Region'] == 'Gambia'),'Country/Region']='Gambia, The'
co2_transport.loc[(co2_industrial_process['Country/Region'] == 'Iran'),'Country/Region']='Iran, Islamic Rep.'
co2_transport.loc[(co2_industrial_process['Country/Region'] == 'Kyrgyzstan'),'Country/Region']='Kyrgyz Republic'
co2_transport.loc[(co2_industrial_process['Country/Region'] == 'Laos'),'Country/Region']='Lao PDR'
co2_transport.loc[(co2_industrial_process['Country/Region'] == 'Micronesia'),'Country/Region']='Micronesia, Fed. Sts.'
co2_transport.loc[(co2_industrial_process['Country/Region'] == 'Macedonia'),'Country/Region']='North Macedonia'
co2_transport.loc[(co2_industrial_process['Country/Region'] == 'North Korea'),'Country/Region']="Korea, Dem. People's Rep."
co2_transport.loc[(co2_industrial_process['Country/Region'] == 'Republic of Congo'),'Country/Region']='Congo, Rep.'
co2_transport.loc[(co2_industrial_process['Country/Region'] == 'Russia'),'Country/Region']='Russian Federation'
co2_transport.loc[(co2_industrial_process['Country/Region'] == 'Saint Kitts and Nevis'),'Country/Region']='St. Kitts and Nevis'
co2_transport.loc[(co2_industrial_process['Country/Region'] == 'Saint Vincent and the Grenadines'),'Country/Region']='St. Vincent and the Grenadines'
co2_transport.loc[(co2_industrial_process['Country/Region'] == 'Slovakia'),'Country/Region']='Slovak Republic'
co2_transport.loc[(co2_industrial_process['Country/Region'] == 'South Korea'),'Country/Region']='Korea, Rep.'
co2_transport.loc[(co2_industrial_process['Country/Region'] == 'Syria'),'Country/Region']='Syrian Arab Republic'
co2_transport.loc[(co2_industrial_process['Country/Region'] == 'Venezuela'),'Country/Region']='Venezuela, RB'
co2_transport.loc[(co2_industrial_process['Country/Region'] == 'Yemen'),'Country/Region']='Yemen, Rep.'

In [12]:
co2_energy_total.loc[(co2_industrial_process['Country/Region'] == 'Bahamas'),'Country/Region']='Bahamas, The'
co2_energy_total.loc[(co2_industrial_process['Country/Region'] == 'Brunei'),'Country/Region']='Brunei Darussalam'
co2_energy_total.loc[(co2_industrial_process['Country/Region'] == 'Cape Verde'),'Country/Region']='Cabo Verde'
co2_energy_total.loc[(co2_industrial_process['Country/Region'] == "CÃ´te d'Ivoire"),'Country/Region']="Cote d'Ivoire"
co2_energy_total.loc[(co2_industrial_process['Country/Region'] == 'Democratic Republic of the Congo'),'Country/Region']='Congo, Dem. Rep.'
co2_energy_total.loc[(co2_industrial_process['Country/Region'] == 'Egypt'),'Country/Region']='Egypt, Arab Rep.'
co2_energy_total.loc[(co2_industrial_process['Country/Region'] == 'Gambia'),'Country/Region']='Gambia, The'
co2_energy_total.loc[(co2_industrial_process['Country/Region'] == 'Iran'),'Country/Region']='Iran, Islamic Rep.'
co2_energy_total.loc[(co2_industrial_process['Country/Region'] == 'Kyrgyzstan'),'Country/Region']='Kyrgyz Republic'
co2_energy_total.loc[(co2_industrial_process['Country/Region'] == 'Laos'),'Country/Region']='Lao PDR'
co2_energy_total.loc[(co2_industrial_process['Country/Region'] == 'Micronesia'),'Country/Region']='Micronesia, Fed. Sts.'
co2_energy_total.loc[(co2_industrial_process['Country/Region'] == 'Macedonia'),'Country/Region']='North Macedonia'
co2_energy_total.loc[(co2_industrial_process['Country/Region'] == 'North Korea'),'Country/Region']="Korea, Dem. People's Rep."
co2_energy_total.loc[(co2_industrial_process['Country/Region'] == 'Republic of Congo'),'Country/Region']='Congo, Rep.'
co2_energy_total.loc[(co2_industrial_process['Country/Region'] == 'Russia'),'Country/Region']='Russian Federation'
co2_energy_total.loc[(co2_industrial_process['Country/Region'] == 'Saint Kitts and Nevis'),'Country/Region']='St. Kitts and Nevis'
co2_energy_total.loc[(co2_industrial_process['Country/Region'] == 'Saint Vincent and the Grenadines'),'Country/Region']='St. Vincent and the Grenadines'
co2_energy_total.loc[(co2_industrial_process['Country/Region'] == 'Slovakia'),'Country/Region']='Slovak Republic'
co2_energy_total.loc[(co2_industrial_process['Country/Region'] == 'South Korea'),'Country/Region']='Korea, Rep.'
co2_energy_total.loc[(co2_industrial_process['Country/Region'] == 'Syria'),'Country/Region']='Syrian Arab Republic'
co2_energy_total.loc[(co2_industrial_process['Country/Region'] == 'Venezuela'),'Country/Region']='Venezuela, RB'
co2_energy_total.loc[(co2_industrial_process['Country/Region'] == 'Yemen'),'Country/Region']='Yemen, Rep.'

### (B) Urban population, economic growth, share of manufacturing sector in an economy, imports and exports

These factors are considered to have impact on carbon dioxide emission based on our findings from literature review.  The data is obtained from World Bank platform which consolidates information from various international organizations.  

For GDP and share of manufacturing sector, figures from United Nations Statistics Division are also used as supplement because relevant time series from World Bank are not so complete for selected countries, such as Eastern European countries before 1990s.

#### (B1) Total population and urban population 
 
* Total population (sourced from United Nations Population Division, expressed in number):
https://data.worldbank.org/indicator/SP.POP.TOTL

* Urban population (sourced from United Nations Population Division, expressed in percentage share of total population):
https://data.worldbank.org/indicator/SP.URB.TOTL.IN.ZS

#### (B2) Economic growth

* GDP at current dollars (expressed in current $US): https://data.worldbank.org/indicator/NY.GDP.MKTP.CD and https://unstats.un.org/unsd/snaama/Basic

* GDP at constant dollars (expressed in 2015 $US): https://data.worldbank.org/indicator/NY.GDP.MKTP.KD and https://unstats.un.org/unsd/snaama/Basic

#### (B3) Share of manufacturing sector in an economy

* Manufacturing, value added (expressed in current $US): https://data.worldbank.org/indicator/NV.IND.MANF.CD and https://unstats.un.org/unsd/snaama/Basic

Note: Value added refers to the 'net output' of an economic activity, which is estimated by deducting intermediate consumption (e.g. cost of raw materials, electricity and water expenses) from gross output (e.g. business receipts).  Sum of value added of all economic activities will be equal to the GDP of the economy concerned.

#### (B4) Imports and exports

* Imports of goods and services (expressed in current $US):
https://data.worldbank.org/indicator/NE.IMP.GNFS.CD

* Merchandise imports (sourced from World Trade Organization, expressed in current $US):
https://data.worldbank.org/indicator/TM.VAL.MRCH.CD.WT

* Exports of goods and services (expressed in current $US):
https://data.worldbank.org/indicator/NE.EXP.GNFS.CD

* Merchandise exports (sourced from World Trade Organization, expressed in current $US):
https://data.worldbank.org/indicator/TX.VAL.MRCH.CD.WT

* Manufactures exports (expressed in percentage share of merchandise exports):
https://data.worldbank.org/indicator/TX.VAL.MANF.ZS.UN

In [13]:
# # Datasets from World Bank
# population = pd.read_sql(path+'API_SP.POP.TOTL_DS2_en_csv_v2_3158886.csv') # Population
# current_gdp1 = pd.read_sql(path+'API_NY.GDP.MKTP.CD_DS2_en_csv_v2_3158872.csv') # GDP in current US$
# constant_gdp1 = pd.read_sql(path+'API_NY.GDP.MKTP.KD_DS2_en_csv_v2_3158988.csv') # GDP in constant 2010 US$
# manufacturing_va1 = pd.read_sql(path+'API_NV.IND.MANF.CD_DS2_en_csv_v2_3165089.csv') # Manufacturing value added in current US$
# medium_high_tech = pd.read_sql(path+'API_NV.MNF.TECH.ZS.UN_DS2_en_csv_v2_3165104.csv') # % of medium to high tech industries, in % manufacturing value added
# export = pd.read_sql(path+'API_NE.EXP.GNFS.CD_DS2_en_csv_v2_3162435.csv') # Export of goods and services in current US$
# imprt = pd.read_sql(path+'API_NE.IMP.GNFS.CD_DS2_en_csv_v2_3159083.csv') # Import of goods and services in current US$
# rgdp_growth = pd.read_sql(path+'API_NY.GDP.MKTP.KD.ZG_DS2_en_csv_v2_3158928.csv') # Growth rate (%) in GDP in constant 2010 US$
# energy_use = pd.read_sql(path+'API_EG.USE.PCAP.KG.OE_DS2_en_csv_v2_3164405.csv') # Energy use per capita in kg of oil equivalent
# urban = pd.read_sql(path+'API_SP.URB.TOTL.IN.ZS_DS2_en_csv_v2_3159585.csv') # Percentage of urban population
# merchandise_export=pd.read_sql(path+'API_TX.VAL.MRCH.CD.WT_DS2_en_csv_v2_3159045.csv') # Export of merchandise goods in current US$
# merchandise_import=pd.read_sql(path+'API_TM.VAL.MRCH.CD.WT_DS2_en_csv_v2_3159105.csv') # Import of merchandise goods in current US$
# manufacturer_export_share=pd.read_sql(path+'API_TX.VAL.MANF.ZS.UN_DS2_en_csv_v2_3165819.csv') # Share of manufactuer export (over merchandise export at current price)
# industrial_va = pd.read_sql(path+'API_NV.IND.TOTL.CD_DS2_en_csv_v2_3165095.csv') # Industrial sector (incl. construction) value added in current US$

# Datasets from World Bank
population = pd.read_sql('api_sp_pop_totl', db.config) # Population
current_gdp1 = pd.read_sql('api_ny_gdp_mktp_cd', db.config) # GDP in current US$
constant_gdp1 = pd.read_sql('api_ny_gdp_mktp_kd', db.config) # GDP in constant 2010 US$
manufacturing_va1 = pd.read_sql('api_nv_ind_manf_cd', db.config) # Manufacturing value added in current US$
medium_high_tech = pd.read_sql('api_nv_mnf_tech_zs_un', db.config) # % of medium to high tech industries, in % manufacturing value added
export = pd.read_sql('api_ne_exp_gnfs_cd', db.config) # Export of goods and services in current US$
imprt = pd.read_sql('api_ne_imp_gnfs_cd', db.config) # Import of goods and services in current US$
rgdp_growth = pd.read_sql('api_ny_gdp_mktp_kd', db.config) # Growth rate (%) in GDP in constant 2010 US$
energy_use = pd.read_sql('api_eg_use_pcap_kg_oe', db.config) # Energy use per capita in kg of oil equivalent
urban = pd.read_sql('api_sp_urb_totl_in_zs', db.config) # Percentage of urban population
merchandise_export=pd.read_sql('api_tx_val_mrch_cd_wt', db.config) # Export of merchandise goods in current US$
merchandise_import=pd.read_sql('api_tm_val_mrch_cd_wt', db.config) # Import of merchandise goods in current US$
manufacturer_export_share=pd.read_sql('api_tx_val_manf_zs_un', db.config) # Share of manufactuer export (over merchandise export at current price)
industrial_va = pd.read_sql('api_nv_ind_totl_cd', db.config) # Industrial sector (incl. construction) value added in current US$


population_world = population.iloc[259,4:].tolist()
current_world_gdp = current_gdp1.iloc[259,4:].tolist()[0:10]
constant_world_gdp = constant_gdp1.iloc[259,4:].tolist()[0:10]
current_world_gdp_2020 = current_gdp1.iloc[259,4:].tolist()[-1]
constant_world_gdp_2020 = constant_gdp1.iloc[259,4:].tolist()[-1]
rgdp_world_growth = rgdp_growth.iloc[259,4:].tolist()
energy_use_world = energy_use.iloc[259,4:].tolist()
urban_world = urban.iloc[259,4:].tolist()
merchandise_export_world = merchandise_export.iloc[259,4:].tolist()
merchandise_import_world = merchandise_import.iloc[259,4:].tolist()
manufactuer_export_share_world = manufacturer_export_share.iloc[259,4:].tolist()
industrial_va_world = industrial_va.iloc[259,4:].tolist()

not_use=['AFE', 'AFW', 'ARB', 'CEB', 'CSS', 'EAP', 'EAR', 'EAS', 'ECA', 'ECS', 'EMU', 'EUU', 'FCS', 'HIC', 
         'HPC', 'IBD', 'IBT', 'IDA', 'IDB', 'IDX', 'INX', 'LAC', 'LCN', 'LDC', 'LIC', 'LMC', 'LMY', 'LTE', 
         'MEA', 'MIC', 'MNA', 'NAC', 'OED', 'OSS', 'PRE', 'PSS', 'PST', 'SAS', 'SSA', 'SSF', 'SST', 'TEA', 
         'TEC', 'TLA', 'TMN', 'TSA', 'TSS', 'UMC', 'WLD']
population = population[~population['Country Code'].isin(not_use)]
current_gdp1 = current_gdp1[~current_gdp1['Country Code'].isin(not_use)]
constant_gdp1 = constant_gdp1[~constant_gdp1['Country Code'].isin(not_use)]
manufacturing_va1 = manufacturing_va1[~manufacturing_va1['Country Code'].isin(not_use)]
medium_high_tech = medium_high_tech[~medium_high_tech['Country Code'].isin(not_use)]
export = export[~export['Country Code'].isin(not_use)]
imprt = imprt[~imprt['Country Code'].isin(not_use)]
rgdp_growth = rgdp_growth[~rgdp_growth['Country Code'].isin(not_use)]
energy_use = energy_use[~energy_use['Country Code'].isin(not_use)]
urban = urban[~urban['Country Code'].isin(not_use)]
merchandise_export = merchandise_export[~merchandise_export['Country Code'].isin(not_use)]
merchandise_import = merchandise_import[~merchandise_import['Country Code'].isin(not_use)]
manufacturer_export_share = manufacturer_export_share[~manufacturer_export_share['Country Code'].isin(not_use)]
industrial_va = industrial_va[~industrial_va['Country Code'].isin(not_use)]

Convert the datasets to long format and merged

In [14]:
# Convert World Bank datasets to long format 
population.drop(population.columns[[2,3]], axis=1, inplace=True)
col_year_list = list(population.columns.values)
col_year_list = col_year_list[2:]
for i in col_year_list:
    population[i] = population[i].astype(float)
population=pd.melt(population, id_vars=['Country Name','Country Code'], value_vars=col_year_list) 
population.rename(columns={'variable':'Year','value':'Population'}, inplace=True)
population['Year'] = population['Year'].astype(int)
population['Country Name'] = population['Country Name'].str.strip() #strip leading blanks

current_gdp1.drop(current_gdp1.columns[[2,3]], axis=1, inplace=True)
for i in col_year_list:
    current_gdp1[i] = current_gdp1[i].astype(float)
current_gdp1=pd.melt(current_gdp1, id_vars=['Country Name','Country Code'], value_vars=col_year_list) 
current_gdp1.rename(columns={'variable':'Year','value':'Current GDP'}, inplace=True)
current_gdp1['Year'] = current_gdp1['Year'].astype(int)
current_gdp1['Country Name'] = current_gdp1['Country Name'].str.strip() #strip leading blanks

constant_gdp1.drop(constant_gdp1.columns[[2,3]], axis=1, inplace=True)
for i in col_year_list:
    constant_gdp1[i] = constant_gdp1[i].astype(float)
constant_gdp1=pd.melt(constant_gdp1, id_vars=['Country Name','Country Code'], value_vars=col_year_list) 
constant_gdp1.rename(columns={'variable':'Year','value':'Constant GDP'}, inplace=True)
constant_gdp1['Year'] = constant_gdp1['Year'].astype(int)
constant_gdp1['Country Name'] = constant_gdp1['Country Name'].str.strip() #strip leading blanks

manufacturing_va1.drop(manufacturing_va1.columns[[2,3]], axis=1, inplace=True)
for i in col_year_list:
    manufacturing_va1[i] = manufacturing_va1[i].astype(float)
manufacturing_va1=pd.melt(manufacturing_va1, id_vars=['Country Name','Country Code'], value_vars=col_year_list) 
manufacturing_va1.rename(columns={'variable':'Year','value':'Manufacturing GDP'}, inplace=True)
manufacturing_va1['Year'] = manufacturing_va1['Year'].astype(int)
manufacturing_va1['Country Name'] = manufacturing_va1['Country Name'].str.strip() #strip leading blanks

medium_high_tech.drop(medium_high_tech.columns[[2,3]], axis=1, inplace=True)
for i in col_year_list:
    medium_high_tech[i] = medium_high_tech[i].astype(float)
medium_high_tech=pd.melt(medium_high_tech, id_vars=['Country Name','Country Code'], value_vars=col_year_list) 
medium_high_tech.rename(columns={'variable':'Year','value':'Medium to High Tech %'}, inplace=True)
medium_high_tech['Year'] = medium_high_tech['Year'].astype(int)
medium_high_tech['Country Name'] = medium_high_tech['Country Name'].str.strip() #strip leading blanks

export.drop(export.columns[[2,3]], axis=1, inplace=True)
for i in col_year_list:
    export[i] = export[i].astype(float)
export=pd.melt(export, id_vars=['Country Name','Country Code'], value_vars=col_year_list) 
export.rename(columns={'variable':'Year','value':'Export'}, inplace=True)
export['Year'] = export['Year'].astype(int)
export['Country Name'] = export['Country Name'].str.strip() #strip leading blanks

imprt.drop(imprt.columns[[2,3]], axis=1, inplace=True)
for i in col_year_list:
    imprt[i] = imprt[i].astype(float)
imprt=pd.melt(imprt, id_vars=['Country Name','Country Code'], value_vars=col_year_list) 
imprt.rename(columns={'variable':'Year','value':'Import'}, inplace=True)
imprt['Year'] = imprt['Year'].astype(int)
imprt['Country Name'] = imprt['Country Name'].str.strip() #strip leading blanks

rgdp_growth.drop(rgdp_growth.columns[[2,3]], axis=1, inplace=True)
for i in col_year_list:
    rgdp_growth[i] = rgdp_growth[i].astype(float)
rgdp_growth=pd.melt(rgdp_growth, id_vars=['Country Name','Country Code'], value_vars=col_year_list) 
rgdp_growth.rename(columns={'variable':'Year','value':'Real GDP Growth %'}, inplace=True)
rgdp_growth['Year'] = rgdp_growth['Year'].astype(int)
rgdp_growth['Country Name'] = rgdp_growth['Country Name'].str.strip() #strip leading blanks

energy_use.drop(energy_use.columns[[2,3]], axis=1, inplace=True)
for i in col_year_list:
    energy_use[i] = energy_use[i].astype(float)
energy_use=pd.melt(energy_use, id_vars=['Country Name','Country Code'], value_vars=col_year_list) 
energy_use.rename(columns={'variable':'Year','value':'Energy Use per capita'}, inplace=True)
energy_use['Year'] = energy_use['Year'].astype(int)
energy_use['Country Name'] = energy_use['Country Name'].str.strip() #strip leading blanks

urban.drop(urban.columns[[2,3]], axis=1, inplace=True)
for i in col_year_list:
    urban[i] = urban[i].astype(float)
urban=pd.melt(urban, id_vars=['Country Name','Country Code'], value_vars=col_year_list) 
urban.rename(columns={'variable':'Year','value':'Urban Population Percent'}, inplace=True)
urban['Year'] = urban['Year'].astype(int)
urban['Country Name'] = urban['Country Name'].str.strip() #strip leading blanks

merchandise_export.drop(merchandise_export.columns[[2,3]], axis=1, inplace=True)
for i in col_year_list:
    merchandise_export[i] = merchandise_export[i].astype(float)
merchandise_export=pd.melt(merchandise_export, id_vars=['Country Name','Country Code'], value_vars=col_year_list) 
merchandise_export.rename(columns={'variable':'Year','value':'Merchandise Export'}, inplace=True)
merchandise_export['Year'] = merchandise_export['Year'].astype(int)
merchandise_export['Country Name'] = merchandise_export['Country Name'].str.strip() #strip leading blanks

merchandise_import.drop(merchandise_import.columns[[2,3]], axis=1, inplace=True)
for i in col_year_list:
    merchandise_import[i] = merchandise_import[i].astype(float)
merchandise_import=pd.melt(merchandise_import, id_vars=['Country Name','Country Code'], value_vars=col_year_list) 
merchandise_import.rename(columns={'variable':'Year','value':'Merchandise Import'}, inplace=True)
merchandise_import['Year'] = merchandise_import['Year'].astype(int)
merchandise_import['Country Name'] = merchandise_import['Country Name'].str.strip() #strip leading blanks

manufacturer_export_share.drop(manufacturer_export_share.columns[[2,3]], axis=1, inplace=True)
for i in col_year_list:
    manufacturer_export_share[i] = manufacturer_export_share[i].astype(float)
manufacturer_export_share=pd.melt(manufacturer_export_share, id_vars=['Country Name','Country Code'], value_vars=col_year_list) 
manufacturer_export_share.rename(columns={'variable':'Year','value':'Manufacturer Export Share'}, inplace=True)
manufacturer_export_share['Year'] = manufacturer_export_share['Year'].astype(int)
manufacturer_export_share['Country Name'] = manufacturer_export_share['Country Name'].str.strip() #strip leading blanks

industrial_va.drop(industrial_va.columns[[2,3]], axis=1, inplace=True)
for i in col_year_list:
    industrial_va[i] = industrial_va[i].astype(float)
industrial_va=pd.melt(industrial_va, id_vars=['Country Name','Country Code'], value_vars=col_year_list) 
industrial_va.rename(columns={'variable':'Year','value':'Industrial GDP'}, inplace=True)
industrial_va['Year'] = industrial_va['Year'].astype(int)
industrial_va['Country Name'] = industrial_va['Country Name'].str.strip() #strip leading blanks

In [15]:
# Read UN datasets

# After downloading from UN website, manual adjustments were made to break up former USSR, Yugoslavia, Czechoslovakia data 
# from 1970 to 1989 into the classification of current countries (e.g. Estonia, Latvia, Crotia, Czech republic, etc) on pro-rata basis.  

current_gdp2 = pd.read_sql('gdp_cd_un', db.config) # GDP in current US$ from UN
current_world_gdp2=current_gdp2[current_gdp2['Country/Area']=='World']
current_world_gdp2=current_world_gdp2['GDP, at current prices - US Dollars'].tolist()
for item in current_world_gdp2:
    current_world_gdp.append(float(item))
current_world_gdp.append(current_world_gdp_2020)    
not_use2=['Africa','Americas','Asia','Australia and New Zealand','Caribbean','Central America','Central Asia','Eastern Africa',
          'Eastern Asia','Eastern Europe','Europe','Latin America and the Caribbean','Middle Africa','Northern Africa','Oceania',
          'South America','South-Eastern Asia','Southern Africa','Southern Europe','Sub-Saharan Africa','Western Africa','Western Europe',
          'World','Former Czechoslovakia','Former Ethiopia','Former Netherlands Antilles','Former Sudan','Former USSR','Northern America',
          'Northern Europe','Former Yugoslavia','Southern Asia','Western Asia','Yemen: Former Democratic Yemen','Yemen: Former Yemen Arab Republic']
current_gdp2 = current_gdp2[~current_gdp2['Country/Area'].isin(not_use2)]
country_code_un = ['AFG','ALB','DZA','AND','AGO','AIA','ATG','ARG','ARM','ABW','AUS','AUT','AZE','BHS','BHR','BGD','BRB','BLR',
                   'BEL','BLZ','BEN','BMU','BTN','BOL','BIH','BWA','BRA','VGB','BRN','BGR','BFA','BDI','CPV','KHM','CMR','CAN',
                   'CYM','CAF','TCD','CHL','HKG','MAC','CHN','COL','COM','COG','COK','CRI','HRV','CUB','CUW','CYP','CZE','CIV',
                   'PRK','COD','DNK','DJI','DMA','DOM','ECU','EGY','SLV','GNQ','ERI','EST','ETH','FJI','FIN','FRA','PYF','GAB',
                   'GMB','GEO','DEU','GHA','GRC','GRL','GRD','GTM','GIN','GNB','GUY','HTI','HND','HUN','ISL','IND','IDN','IRN',
                   'IRQ','IRL','ISR','ITA','JAM','JPN','JOR','KAZ','KEN','SWZ','KIR','XKX','KWT','KGZ','LAO','LVA','LBN','LSO',
                   'LBR','LBY','LIE','LTU','LUX','MDG','MWI','MYS','MDV','MLI','MLT','MHL','MRT','MUS','XX1','MEX','XX4','FSM','MCO',
                   'MNG','MNE','MSR','MAR','MOZ','MMR','NAM','NRU','NPL','NLD','NCL','NZL','NIC','NER','NGA','NOR','OMN','PAK',
                   'PLW','PAN','PNG','PRY','PER','PHL','POL','XX2','PRT','PRI','QAT','KOR','MDA','MKD','ROU','RUS','RWA','KNA',
                   'LCA','VCT','WSM','SMR','STP','SAU','SEN','SRB','SYC','SLE','SGP','SXM','SVK','SVN','SLB','SOM','ZAF','SSD',
                   'ESP','LKA','PSE','SDN','SUR','SWE','CHE','SYR','TJK','THA','TLS','TGO','TON','TTO','TUN','TUR','TKM','TCA',
                   'TUV','UGA','UKR','ARE','GBR','TZA','XX3','USA','URY','UZB','VUT','VEN','VNM','YEM','ZMB','ZWE'
                  ]
from itertools import chain, repeat
country_lst_un=list(chain.from_iterable(zip(*repeat(country_code_un, 50))))
current_gdp2['Country Code']=country_lst_un
current_gdp2.drop(['Country/Area','Unit'],axis=1,inplace=True)
current_gdp2.rename(columns={'GDP, at current prices - US Dollars': 'Current GDP UN'}, inplace=True)
current_gdp=pd.merge(current_gdp1,current_gdp2,how='left',on=['Country Code','Year'])
current_gdp['Current GDP UN'] = np.where(~current_gdp['Current GDP UN'].isnull(),current_gdp['Current GDP UN'],current_gdp['Current GDP'])
current_gdp.drop(['Country Name','Current GDP'],axis=1,inplace=True)
current_gdp.rename(columns={'Current GDP UN': 'Current GDP'}, inplace=True)
current_gdp['Current GDP'] = current_gdp['Current GDP'].replace('...', np.nan)
current_gdp = current_gdp.astype({"Current GDP": float})

constant_gdp2 = pd.read_sql('gdp_kd_un', db.config) # GDP in constant 2015 US$ from UN
constant_world_gdp2=constant_gdp2[constant_gdp2['Country/Area']=='World']
constant_world_gdp2=constant_world_gdp2['GDP, at constant 2015 prices - US Dollars'].tolist()
for item in constant_world_gdp2:
    constant_world_gdp.append(float(item))
constant_world_gdp.append(constant_world_gdp_2020)    
constant_gdp2 = constant_gdp2[~constant_gdp2['Country/Area'].isin(not_use2)]
constant_gdp2['Country Code']=country_lst_un
constant_gdp2.drop(['Country/Area','Unit'],axis=1,inplace=True)
constant_gdp2.rename(columns={'GDP, at constant 2015 prices - US Dollars': 'Constant GDP UN'}, inplace=True)
constant_gdp=pd.merge(constant_gdp1,constant_gdp2,how='left',on=['Country Code','Year'])
constant_gdp['Constant GDP UN'] = np.where(~constant_gdp['Constant GDP UN'].isnull(),constant_gdp['Constant GDP UN'],constant_gdp['Constant GDP'])
constant_gdp.drop(['Country Name','Constant GDP'],axis=1,inplace=True)
constant_gdp.rename(columns={'Constant GDP UN': 'Constant GDP'}, inplace=True)
constant_gdp['Constant GDP'] = constant_gdp['Constant GDP'].replace('...', np.nan)
constant_gdp = constant_gdp.astype({"Constant GDP": float})

manufacturing_va2 = pd.read_sql('manufacturing_va_cd_un', db.config) # Value added of manufacturing from UN
manufacturing_va2 = manufacturing_va2[~manufacturing_va2['Country/Area'].isin(not_use2)]
manufacturing_va2['Country Code']=country_lst_un
manufacturing_va2.drop(['Country/Area','Unit','Agriculture, hunting, forestry, fishing (ISIC A-B)',
                       'Mining, Manufacturing, Utilities (ISIC C-E)','Construction (ISIC F)','Wholesale, retail trade, restaurants and hotels (ISIC G-H)',
                       'Transport, storage and communication (ISIC I)','Other Activities (ISIC J-P)','Total Value Added'],axis=1,inplace=True)
manufacturing_va2.rename(columns={'Manufacturing (ISIC D)': 'Manufacturing GDP UN'}, inplace=True)
manufacturing_va=pd.merge(manufacturing_va1,manufacturing_va2,how='left',on=['Country Code','Year'])
manufacturing_va['Manufacturing GDP UN'] = np.where(~manufacturing_va['Manufacturing GDP UN'].isnull(),manufacturing_va['Manufacturing GDP UN'],manufacturing_va['Manufacturing GDP'])
manufacturing_va.drop(['Country Name','Manufacturing GDP'],axis=1,inplace=True)
manufacturing_va.rename(columns={'Manufacturing GDP UN': 'Manufacturing GDP'}, inplace=True)
manufacturing_va['Manufacturing GDP'] = manufacturing_va['Manufacturing GDP'].replace('...', np.nan)
manufacturing_va = manufacturing_va.astype({"Manufacturing GDP": float})

In [16]:
# Convert Climate Watch datasets to long format
co2_electricity.drop(co2_electricity.columns[[1]], axis=1, inplace=True)
co2_electricity.replace('FALSE', np.nan, inplace=True)
col_year_list2 = list(co2_electricity.columns.values)
col_year_list2 = col_year_list2[1:]
for i in col_year_list2:
    co2_electricity[i] = co2_electricity[i].astype(float)
co2_electricity=pd.melt(co2_electricity, id_vars=['Country/Region'], value_vars=col_year_list2) 
co2_electricity.rename(columns={'variable':'Year','value':'CO2 Emission_Electricity','Country/Region':'Country Name'}, inplace=True)
co2_electricity['Year'] = co2_electricity['Year'].astype(int)
co2_electricity['Country Name'] = co2_electricity['Country Name'].str.strip() #strip leading blanks
co2_electricity['CO2 Emission_Electricity']=co2_electricity['CO2 Emission_Electricity']*1000 #convert from Mt to kt

co2_building.drop(co2_building.columns[[1]], axis=1, inplace=True)
co2_building.replace('FALSE', np.nan, inplace=True)
for i in col_year_list2:
    co2_building[i] = co2_building[i].astype(float)
co2_building=pd.melt(co2_building, id_vars=['Country/Region'], value_vars=col_year_list2) 
co2_building.rename(columns={'variable':'Year','value':'CO2 Emission_Building','Country/Region':'Country Name'}, inplace=True)
co2_building['Year'] = co2_building['Year'].astype(int)
co2_building['Country Name'] = co2_building['Country Name'].str.strip() #strip leading blanks
co2_building['CO2 Emission_Building']=co2_building['CO2 Emission_Building']*1000 #convert from Mt to kt

co2_manufacturing.drop(co2_manufacturing.columns[[1]], axis=1, inplace=True)
co2_manufacturing.replace('FALSE', np.nan, inplace=True)
for i in col_year_list2:
    co2_manufacturing[i] = co2_manufacturing[i].astype(float)
co2_manufacturing=pd.melt(co2_manufacturing, id_vars=['Country/Region'], value_vars=col_year_list2) 
co2_manufacturing.rename(columns={'variable':'Year','value':'CO2 Emission_Manufacturing','Country/Region':'Country Name'}, inplace=True)
co2_manufacturing['Year'] = co2_manufacturing['Year'].astype(int)
co2_manufacturing['Country Name'] = co2_manufacturing['Country Name'].str.strip() #strip leading blanks
co2_manufacturing['CO2 Emission_Manufacturing']=co2_manufacturing['CO2 Emission_Manufacturing']*1000 #convert from Mt to kt

co2_other_fuel.drop(co2_other_fuel.columns[[1]], axis=1, inplace=True)
co2_other_fuel.replace('FALSE', np.nan, inplace=True)
for i in col_year_list2:
    co2_other_fuel[i] = co2_other_fuel[i].astype(float)
co2_other_fuel=pd.melt(co2_other_fuel, id_vars=['Country/Region'], value_vars=col_year_list2) 
co2_other_fuel.rename(columns={'variable':'Year','value':'CO2 Emission_Other_Fuel','Country/Region':'Country Name'}, inplace=True)
co2_other_fuel['Year'] = co2_other_fuel['Year'].astype(int)
co2_other_fuel['Country Name'] = co2_other_fuel['Country Name'].str.strip() #strip leading blanks
co2_other_fuel['CO2 Emission_Other_Fuel']=co2_other_fuel['CO2 Emission_Other_Fuel']*1000 #convert from Mt to kt

co2_fugitive.drop(co2_fugitive.columns[[1]], axis=1, inplace=True)
co2_fugitive.replace('FALSE', np.nan, inplace=True)
for i in col_year_list2:
    co2_fugitive[i] = co2_fugitive[i].astype(float)
co2_fugitive=pd.melt(co2_fugitive, id_vars=['Country/Region'], value_vars=col_year_list2) 
co2_fugitive.rename(columns={'variable':'Year','value':'CO2 Emission_Fugitive','Country/Region':'Country Name'}, inplace=True)
co2_fugitive['Year'] = co2_fugitive['Year'].astype(int)
co2_fugitive['Country Name'] = co2_fugitive['Country Name'].str.strip() #strip leading blanks
co2_fugitive['CO2 Emission_Fugitive']=co2_fugitive['CO2 Emission_Fugitive']*1000 #convert from Mt to kt

co2_bunkers.drop(co2_bunkers.columns[[1]], axis=1, inplace=True)
co2_bunkers.replace('FALSE', np.nan, inplace=True)
for i in col_year_list2:
    co2_bunkers[i] = co2_bunkers[i].astype(float)
co2_bunkers=pd.melt(co2_bunkers, id_vars=['Country/Region'], value_vars=col_year_list2) 
co2_bunkers.rename(columns={'variable':'Year','value':'CO2 Emission_Bunkers','Country/Region':'Country Name'}, inplace=True)
co2_bunkers['Year'] = co2_bunkers['Year'].astype(int)
co2_bunkers['Country Name'] = co2_bunkers['Country Name'].str.strip() #strip leading blanks
co2_bunkers['CO2 Emission_Bunkers']=co2_bunkers['CO2 Emission_Bunkers']*1000 #convert from Mt to kt

co2_industrial_process.drop(co2_industrial_process.columns[[1]], axis=1, inplace=True)
co2_industrial_process.replace('FALSE', np.nan, inplace=True)
for i in col_year_list2:
    co2_industrial_process[i] = co2_industrial_process[i].astype(float)
co2_industrial_process=pd.melt(co2_industrial_process, id_vars=['Country/Region'], value_vars=col_year_list2) 
co2_industrial_process.rename(columns={'variable':'Year','value':'CO2 Emission_Industrial_Process','Country/Region':'Country Name'}, inplace=True)
co2_industrial_process['Year'] = co2_industrial_process['Year'].astype(int)
co2_industrial_process['Country Name'] = co2_industrial_process['Country Name'].str.strip() #strip leading blanks
co2_industrial_process['CO2 Emission_Industrial_Process']=co2_industrial_process['CO2 Emission_Industrial_Process']*1000 #convert from Mt to kt

co2_transport.drop(co2_transport.columns[[1]], axis=1, inplace=True)
co2_transport.replace('FALSE', np.nan, inplace=True)
for i in col_year_list2:
    co2_transport[i] = co2_transport[i].astype(float)
co2_transport=pd.melt(co2_transport, id_vars=['Country/Region'], value_vars=col_year_list2) 
co2_transport.rename(columns={'variable':'Year','value':'CO2 Emission_Transport','Country/Region':'Country Name'}, inplace=True)
co2_transport['Year'] = co2_transport['Year'].astype(int)
co2_transport['Country Name'] = co2_transport['Country Name'].str.strip() #strip leading blanks
co2_transport['CO2 Emission_Transport']=co2_transport['CO2 Emission_Transport']*1000 #convert from Mt to kt

co2_energy_total.drop(co2_energy_total.columns[[1]], axis=1, inplace=True)
co2_energy_total.replace('FALSE', np.nan, inplace=True)
for i in col_year_list2:
    co2_energy_total[i] = co2_energy_total[i].astype(float)
co2_energy_total=pd.melt(co2_energy_total, id_vars=['Country/Region'], value_vars=col_year_list2) 
co2_energy_total.rename(columns={'variable':'Year','value':'CO2 Emission_Energy_Subtotal','Country/Region':'Country Name'}, inplace=True)
co2_energy_total['Year'] = co2_energy_total['Year'].astype(int)
co2_energy_total['Country Name'] = co2_energy_total['Country Name'].str.strip() #strip leading blanks
co2_energy_total['CO2 Emission_Energy_Subtotal']=co2_energy_total['CO2 Emission_Energy_Subtotal']*1000 #convert from Mt to kt

In [17]:
# Merge the CO2 dataset, World Bank and Climate Watch datasets

print(f'Population {population.shape} co2_emission {co2_emission.shape}')
data=pd.merge(population,co2_emission,how='left',on=['Country Code','Year'])
print(f'Data {data.shape}')

data=pd.merge(data,current_gdp,how='left',on=['Country Code','Year'])
print(f'Data {data.shape}')
data=pd.merge(data,constant_gdp,how='left',on=['Country Code','Year'])
print(f'Data {data.shape}')
data=pd.merge(data,manufacturing_va,how='left',on=['Country Code','Year'])
print(f'Data {data.shape}')
data=pd.merge(data,medium_high_tech,how='left',on=['Country Name','Country Code','Year'])
print(f'Data {data.shape}')
data=pd.merge(data,export,how='left',on=['Country Name','Country Code','Year'])
print(f'Data {data.shape}')
data=pd.merge(data,imprt,how='left',on=['Country Name','Country Code','Year'])
print(f'Data {data.shape}')
data=pd.merge(data,rgdp_growth,how='left',on=['Country Name','Country Code','Year'])
print(f'Data {data.shape}')
data=pd.merge(data,energy_use,how='left',on=['Country Name','Country Code','Year'])
print(f'Data {data.shape}')
data=pd.merge(data,urban,how='left',on=['Country Name','Country Code','Year'])
print(f'Data {data.shape}')
data=pd.merge(data,merchandise_export,how='left',on=['Country Name','Country Code','Year'])
print(f'Data {data.shape}')
data=pd.merge(data,merchandise_import,how='left',on=['Country Name','Country Code','Year'])
print(f'Data {data.shape}')
data=pd.merge(data,manufacturer_export_share,how='left',on=['Country Name','Country Code','Year'])
print(f'Data {data.shape}')
data=pd.merge(data,industrial_va,how='left',on=['Country Name','Country Code','Year'])
print(f'Data {data.shape}')

Population (13237, 4) co2_emission (12772, 3)
Data (13237, 5)
Data (13243, 6)
Data (13254, 7)
Data (13335, 8)
Data (13335, 9)
Data (13335, 10)
Data (13335, 11)
Data (13335, 12)
Data (13335, 13)
Data (13335, 14)
Data (13335, 15)
Data (13335, 16)
Data (13335, 17)
Data (13335, 18)


In [18]:
data=pd.merge(data, co2_electricity,how='left',on=['Country Name','Year'])
data=pd.merge(data, co2_building,how='left',on=['Country Name','Year'])
data=pd.merge(data, co2_manufacturing,how='left',on=['Country Name','Year'])
data=pd.merge(data, co2_other_fuel,how='left',on=['Country Name','Year'])
data=pd.merge(data, co2_fugitive,how='left',on=['Country Name','Year'])
data=pd.merge(data, co2_transport,how='left',on=['Country Name','Year'])
data=pd.merge(data, co2_energy_total,how='left',on=['Country Name','Year'])
data=pd.merge(data, co2_bunkers,how='left',on=['Country Name','Year'])
data=pd.merge(data, co2_industrial_process,how='left',on=['Country Name','Year'])

In [19]:
# Add missing constant GDP of Canada from 1960 to 1969 (based on yoy change in constant GDP obtained from International Monetary Fund and St Louis Fred
# https://www.imf.org/en/Publications/WEO/weo-database/2021/April/download-entire-database)
# https://fred.stlouisfed.org/series/NAEXKP01CAA657S
data.loc[data.index[(data['Country Code']=='CAN')&(data['Year']==1969)][0],'Constant GDP']=data.loc[data.index[(data['Country Code']=='CAN')&(data['Year']==1970)][0],'Constant GDP']/(1+0.0296)
data.loc[data.index[(data['Country Code']=='CAN')&(data['Year']==1968)][0],'Constant GDP']=data.loc[data.index[(data['Country Code']=='CAN')&(data['Year']==1969)][0],'Constant GDP']/(1+0.05046)
data.loc[data.index[(data['Country Code']=='CAN')&(data['Year']==1967)][0],'Constant GDP']=data.loc[data.index[(data['Country Code']=='CAN')&(data['Year']==1968)][0],'Constant GDP']/(1+0.04996)
data.loc[data.index[(data['Country Code']=='CAN')&(data['Year']==1966)][0],'Constant GDP']=data.loc[data.index[(data['Country Code']=='CAN')&(data['Year']==1967)][0],'Constant GDP']/(1+0.03091)
data.loc[data.index[(data['Country Code']=='CAN')&(data['Year']==1965)][0],'Constant GDP']=data.loc[data.index[(data['Country Code']=='CAN')&(data['Year']==1966)][0],'Constant GDP']/(1+0.06705)
data.loc[data.index[(data['Country Code']=='CAN')&(data['Year']==1964)][0],'Constant GDP']=data.loc[data.index[(data['Country Code']=='CAN')&(data['Year']==1965)][0],'Constant GDP']/(1+0.06317)
data.loc[data.index[(data['Country Code']=='CAN')&(data['Year']==1963)][0],'Constant GDP']=data.loc[data.index[(data['Country Code']=='CAN')&(data['Year']==1964)][0],'Constant GDP']/(1+0.06643)
data.loc[data.index[(data['Country Code']=='CAN')&(data['Year']==1962)][0],'Constant GDP']=data.loc[data.index[(data['Country Code']=='CAN')&(data['Year']==1963)][0],'Constant GDP']/(1+0.0535)
data.loc[data.index[(data['Country Code']=='CAN')&(data['Year']==1961)][0],'Constant GDP']=data.loc[data.index[(data['Country Code']=='CAN')&(data['Year']==1962)][0],'Constant GDP']/(1+0.07425)
data.loc[data.index[(data['Country Code']=='CAN')&(data['Year']==1960)][0],'Constant GDP']=data.loc[data.index[(data['Country Code']=='CAN')&(data['Year']==1961)][0],'Constant GDP']/(1+0.03163)

In [20]:
# Add New Zealand import and export 2020 figures https://www.stats.govt.nz/information-releases/gross-domestic-product-june-2021-quarter
data.loc[data.index[(data['Country Code']=='NZL')&(data['Year']==2020)][0],'Export']=88504*1000000*0.6557
data.loc[data.index[(data['Country Code']=='NZL')&(data['Year']==2020)][0],'Import']=87536*1000000*0.6557

In [21]:
# Add missing current and constant GDP of selected countries
# United Arab Emirates https://www.statista.com/statistics/297605/uae-gross-domestic-product/
data.loc[data.index[(data['Country Code']=='ARE')&(data['Year']==2020)][0],'Current GDP']=354.28*1000000000
data.loc[data.index[(data['Country Code']=='ARE')&(data['Year']==2020)][0],'Constant GDP']=data.loc[data.index[(data['Country Code']=='ARE')&(data['Year']==2019)][0],'Constant GDP']*354.28*1000000000/data.loc[data.index[(data['Country Code']=='ARE')&(data['Year']==2019)][0],'Current GDP']

# Cuba https://www.reuters.com/article/cuba-economy-idUSL1N2IX1V9
data.loc[data.index[(data['Country Code']=='CUB')&(data['Year']==2020)][0],'Constant GDP']=data.loc[data.index[(data['Country Code']=='CUB')&(data['Year']==2019)][0],'Constant GDP']*(1-0.11)
data.loc[data.index[(data['Country Code']=='CUB')&(data['Year']==2020)][0],'Current GDP']=data.loc[data.index[(data['Country Code']=='CUB')&(data['Year']==2019)][0],'Current GDP']*(data.loc[data.index[(data['Country Code']=='CUB')&(data['Year']==2020)][0],'Constant GDP']/data.loc[data.index[(data['Country Code']=='CUB')&(data['Year']==2019)][0],'Constant GDP'])

# Oman https://www.statista.com/statistics/455593/gross-domestic-product-gdp-in-oman/
data.loc[data.index[(data['Country Code']=='OMN')&(data['Year']==2020)][0],'Current GDP']=63.37*1000000000
data.loc[data.index[(data['Country Code']=='OMN')&(data['Year']==2020)][0],'Constant GDP']=data.loc[data.index[(data['Country Code']=='OMN')&(data['Year']==2019)][0],'Constant GDP']*63.37*1000000000/data.loc[data.index[(data['Country Code']=='OMN')&(data['Year']==2019)][0],'Current GDP']

# Venezuela https://knoema.com/atlas/Venezuela-Bolivarian-Republic-of/GDP
data.loc[data.index[(data['Country Code']=='VEN')&(data['Year']==2020)][0],'Constant GDP']=data.loc[data.index[(data['Country Code']=='VEN')&(data['Year']==2019)][0],'Constant GDP']*(1-0.2612)
data.loc[data.index[(data['Country Code']=='VEN')&(data['Year']==2020)][0],'Current GDP']=data.loc[data.index[(data['Country Code']=='VEN')&(data['Year']==2019)][0],'Current GDP']*(data.loc[data.index[(data['Country Code']=='VEN')&(data['Year']==2020)][0],'Constant GDP']/data.loc[data.index[(data['Country Code']=='VEN')&(data['Year']==2019)][0],'Constant GDP'])

# North Korea https://www.bloomberg.com/news/articles/2021-07-30/north-korea-s-economy-contracted-most-in-two-decades-in-2020
data.loc[data.index[(data['Country Code']=='PRK')&(data['Year']==2020)][0],'Constant GDP']=data.loc[data.index[(data['Country Code']=='PRK')&(data['Year']==2019)][0],'Constant GDP']*(1-0.045)
data.loc[data.index[(data['Country Code']=='PRK')&(data['Year']==2020)][0],'Current GDP']=data.loc[data.index[(data['Country Code']=='PRK')&(data['Year']==2019)][0],'Current GDP']*(data.loc[data.index[(data['Country Code']=='PRK')&(data['Year']==2020)][0],'Constant GDP']/data.loc[data.index[(data['Country Code']=='PRK')&(data['Year']==2019)][0],'Constant GDP'])

# Syria https://en.wikipedia.org/wiki/Economy_of_Syria
data.loc[data.index[(data['Country Code']=='SYR')&(data['Year']==2020)][0],'Constant GDP']=data.loc[data.index[(data['Country Code']=='SYR')&(data['Year']==2019)][0],'Constant GDP']*(1-0.03)
data.loc[data.index[(data['Country Code']=='SYR')&(data['Year']==2020)][0],'Current GDP']=data.loc[data.index[(data['Country Code']=='SYR')&(data['Year']==2019)][0],'Current GDP']*(data.loc[data.index[(data['Country Code']=='SYR')&(data['Year']==2020)][0],'Constant GDP']/data.loc[data.index[(data['Country Code']=='SYR')&(data['Year']==2019)][0],'Constant GDP'])

# Turkmenistan https://en.wikipedia.org/wiki/Economy_of_Turkmenistan
data.loc[data.index[(data['Country Code']=='TKM')&(data['Year']==2020)][0],'Current GDP']=47.986*1000000000
data.loc[data.index[(data['Country Code']=='TKM')&(data['Year']==2020)][0],'Constant GDP']=data.loc[data.index[(data['Country Code']=='TKM')&(data['Year']==2019)][0],'Constant GDP']*47.986*1000000000/data.loc[data.index[(data['Country Code']=='TKM')&(data['Year']==2019)][0],'Current GDP']

# Bahrain
data.loc[data.index[(data['Country Code']=='BHR')&(data['Year']==2020)][0],'Current GDP']=data.loc[data.index[(data['Country Code']=='BHR')&(data['Year']==2019)][0],'Current GDP']*data.loc[data.index[(data['Country Code']=='BHR')&(data['Year']==2020)][0],'Constant GDP']/data.loc[data.index[(data['Country Code']=='BHR')&(data['Year']==2019)][0],'Constant GDP']

Based on the merged dataset, the following features which may be useful for our analysis are compiled:
* Carbon dioxide emission per capita (ton per person) = (Carbon dioxide emission * 1000)/Total Population
* Carbon dioxide emission per constant GDP dollar (kg per 2015 USD) = (Carbon dioxide emission * 1000000)/GDP at constant dollars
* Constant GDP per capita (2015 USD per person) = GDP at constant dollars/Total Population
* Manufacturing share (percent) = (Manufacturing, value added/GDP at current dollars) * 100
* Merchandise export share (percent) = (Merchandise exports/GDP at current dollars) * 100
* Merchandise import share (percent) = (Merchandise imports/GDP at current dollars) * 100
* Trade openness (percent) = (Sum of exports and imports of goods and services/GDP at current dollars) * 100

In [22]:
# Compute CO2 emission per capita (metric tonne per person), constant GDP per capita and others
data['CO2 Emission per capita']=data['CO2 Emission']*1000/data['Population'] 
data['Constant GDP per capita']=data['Constant GDP']/data['Population']
data['Manufacturing %']=data['Manufacturing GDP']/data['Current GDP']*100
data['Medium to High Tech GDP']=data['Manufacturing GDP']*data['Medium to High Tech %']/100
data['CO2 Emission per constant GDP']=data['CO2 Emission']*1000000/data['Constant GDP'] # in kg per constant $US GDP
data['Trade Openness']=((data['Export']+data['Import'])/data['Current GDP'])*100
data['Share of Merchandise Export']=data['Merchandise Export']/data['Current GDP']*100
data['Share of Merchandise Import']=data['Merchandise Import']/data['Current GDP']*100
data['Manufacturer Export']=data['Merchandise Export']*(data['Manufacturer Export Share']/100)

In [23]:
col_sequence=['Country Name','Country Code','Year','CO2 Emission','Population','Current GDP','Constant GDP','Manufacturing GDP','Medium to High Tech %','Export','Import',
              'Real GDP Growth %','Energy Use per capita','Urban Population Percent','Merchandise Export','Merchandise Import','Manufacturer Export Share','Manufacturer Export',
              'CO2 Emission_Electricity','CO2 Emission_Building','CO2 Emission_Manufacturing','CO2 Emission_Other_Fuel','CO2 Emission_Fugitive',
              'CO2 Emission_Transport','CO2 Emission_Energy_Subtotal','CO2 Emission_Bunkers','CO2 Emission_Industrial_Process','CO2 Emission per capita',
              'Constant GDP per capita','Manufacturing %','Medium to High Tech GDP','CO2 Emission per constant GDP','Trade Openness',
              'Share of Merchandise Export','Share of Merchandise Import','Industrial GDP']
data=data[col_sequence]

Time series of value added of manufacturing from UN starts from 1970, while time series from World Bank starts even later in 1990. To extend the analysis to a longer period:
* Value added of manufacturing sector in the US from 1960 to 1969 was obtained from Bureau of Economic Analysis https://apps.bea.gov/iTable/iTable.cfm?reqid=147&step=2&isuri=1

* Value added of manufacturing sector in China from 1960 to 2003 was estimated based on value added of industrial sector (which covers both manufacturing and construction sectors and can be found from World Bank)
https://data.worldbank.org/indicator/NV.IND.TOTL.ZS

* For manufacturing value added of other countries, the figures are imputed using changes in manufacture exports, based on an assumption that change in manufacturing output and manufacturing export should be similar for most of the times. i.e., Manufacturing GDP at time t = Manufacturing GDP at time (t-1) * [Manufacture exports at time t / Manufacture exports at time (t-1)]

* Finally, since economic structure only changes over a relatively long time horizon in terms of decades, manufacturing share estimated in 2019 is brought forward to 2020, and value added of manufacturing sector in 2020 is estimated by multiplying the actual current GDP in 2020 by the imputed manufacturing share in percentage.  Similarly, manufacturing share estimated in 1970 is brought forward to period fromd 1960 to 1969, and value added of manufacturing sector in 1960 - 1969 are estimated by multiplying the actual current GDP in 1960 - 1969 by the imputed manufacturing share in percentage.

In [24]:
# Add US manufacturing GDP from 1960 to 1969
path_to_data = r'data/raw/complex'
path = str(pathlib.Path.cwd().parent.joinpath(path_to_data))+r"/"

xls2 = pd.ExcelFile(path+'GDPbyInd_VA_1947-1997.xlsx')
us_manufacturing_gdp = pd.read_excel(xls2, 'VA', skiprows=[5])
us_manufacturing_gdp = us_manufacturing_gdp.iloc[15,15:25].tolist()
year = list(range(1960,1970))
country = ['USA']*10
us_manufacturing_df = pd.DataFrame({'Country Code':country,'Year':year,'Manufacturing GDP US':us_manufacturing_gdp})
us_manufacturing_df['Manufacturing GDP US']=us_manufacturing_df['Manufacturing GDP US']*1000000 #GDP from million $US to $US, current price
new_row = {'Country Code':'USA', 'Year':2020, 'Manufacturing GDP US':2272*1000000000} #2020 manufacturing value added from BEA https://www.bea.gov/data/gdp/gdp-industry
us_manufacturing_df = us_manufacturing_df.append(new_row, ignore_index=True)

data=pd.merge(data,us_manufacturing_df,how='left',on=['Country Code','Year'])
data['Manufacturing GDP']=np.where((data['Country Code']=='USA')&(data['Manufacturing GDP'].isnull()),data['Manufacturing GDP US'],data['Manufacturing GDP'])
data.drop(['Manufacturing GDP US'],axis=1,inplace=True) 

In [25]:
# Add missing manufacturing GDP in China from 1960 to 1969
# Assume 69% of industrial value added belongs to manufacturing based on 2004 situation
data['Manufacturing GDP']=np.where((data['Country Code']=='CHN')&(data['Manufacturing GDP'].isnull()),data['Industrial GDP']*0.69,data['Manufacturing GDP'])

In [26]:
# Impute missing manufacturing GDP in other countries (based on change in manufacture exports)
data=data.sort_values(by=['Country Code','Year'],ascending=False)
data['Country Code_adv1']=data['Country Code'].shift(1)
data['Manufacturer Export_adv1']=data['Manufacturer Export'].shift(1)
data['Manufacturing Growth']=np.where(data['Country Code']==data['Country Code_adv1'],data['Manufacturer Export_adv1']/data['Manufacturer Export']-1,np.nan)
for i in range(1,len(data)):
    if (data.iloc[i,1]==data.iloc[i,36])&(pd.isnull(data.iloc[i,7]))&(pd.notnull(data.iloc[i-1,7]))&(pd.notnull(data.iloc[i,38])):
        data.iloc[i,7]=data.iloc[i-1,7]/(1+data.iloc[i,38])
data.drop(['Country Code_adv1','Manufacturer Export_adv1','Manufacturing Growth'],axis=1,inplace=True)  
data['Manufacturing %']=data['Manufacturing GDP']/data['Current GDP']*100
data=data[col_sequence]

# Finally, apply 2019 manufacturing share to 2020, then derive 2020 manufacturing value added based on 2020 current GDP
# Also apply 1970 manufacturing share to 1960-1969, then derive 1960-1969 manufacturing value added based on 1960-1969 current GDP
codes=data['Country Code'].unique().tolist()
lst1=[]
lst2=[]
lst3=[]
lst4=[]
for code in codes:
    lst1.append(data[data['Country Code']==code]['Manufacturing %'].fillna(method='bfill'))
for item1 in lst1:
    for item2 in item1:
        lst2.append(item2)
data['Manufacturing2 %']=lst2
for code in codes:
    lst3.append(data[data['Country Code']==code]['Manufacturing2 %'].fillna(method='ffill'))
for item1 in lst3:
    for item2 in item1:
        lst4.append(item2)
data['Manufacturing3 %']=lst4
data.drop(['Manufacturing %','Manufacturing2 %'],axis=1,inplace=True)
data.rename(columns={'Manufacturing3 %':'Manufacturing %'}, inplace=True)
data=data[col_sequence]
data['Manufacturing GDP']=np.where(~data['Manufacturing GDP'].isnull(),data['Manufacturing GDP'],data['Current GDP']*data['Manufacturing %']/100)
data=data.sort_values(by=['Year','Country Code']) 

Compute the world total figures of the features mentioned above

In [27]:
print(f"Shape of data: {data.shape}")
# World total emission
data_gp = data.groupby('Year').agg({'Manufacturing GDP':'sum', 'Medium to High Tech GDP':'sum', 'Export':'sum', 
                                    'Import':'sum', 'CO2 Emission_Electricity':'sum',
                                    'CO2 Emission_Building':'sum', 'CO2 Emission_Manufacturing':'sum', 'CO2 Emission_Other_Fuel':'sum',
                                    'CO2 Emission_Fugitive':'sum', 'CO2 Emission_Bunkers':'sum', 'CO2 Emission_Industrial_Process':'sum',
                                    'CO2 Emission_Transport':'sum', 'CO2 Emission_Energy_Subtotal':'sum'
                                   }).reset_index()

print(f"Shape of data_gp: {data_gp.shape}")

data_gp['CO2 Emission']=co2_world_emission
data_gp['Population']=population_world 
data_gp['Current GDP']=current_world_gdp
data_gp['Constant GDP']=constant_world_gdp 
data_gp['Manufacturing %']=data_gp['Manufacturing GDP']/data_gp['Current GDP']*100
data_gp['Medium to High Tech %']=data_gp['Medium to High Tech GDP']/data_gp['Manufacturing GDP']*100
data_gp['CO2 Emission per capita']=data_gp['CO2 Emission']*1000/data_gp['Population'] # metric tonne per person
data_gp['Constant GDP per capita']=data_gp['Constant GDP']/data_gp['Population'] # 2015 $US per person
data_gp['CO2 Emission per constant GDP']=data_gp['CO2 Emission']*1000000/data_gp['Constant GDP'] # in kg per constant GDP $US
data_gp['Country Name']='World'
data_gp['Country Code']='WLD'
data_gp['Real GDP Growth %']=rgdp_world_growth
data_gp['Energy Use per capita']=energy_use_world
data_gp['Urban Population Percent']=urban_world
data_gp['Merchandise Export']=merchandise_export_world
data_gp['Merchandise Import']=merchandise_import_world
data_gp['Industrial GDP']=industrial_va_world
data_gp['Manufacturer Export Share']=manufactuer_export_share_world
data_gp['Trade Openness']=((data_gp['Export']+data_gp['Import'])/data_gp['Current GDP'])*100
data_gp['Share of Merchandise Export']=(data_gp['Merchandise Export']/data_gp['Current GDP'])*100
data_gp['Share of Merchandise Import']=(data_gp['Merchandise Import']/data_gp['Current GDP'])*100
data_gp['Manufacturer Export']=data_gp['Merchandise Export']*(data_gp['Manufacturer Export Share']/100)
data_gp=data_gp[col_sequence]

Shape of data: (13335, 36)
Shape of data_gp: (61, 14)


In [28]:
# Append world total back to data
data2=data.append(data_gp,ignore_index=True)

With the world totals, we can estimate share of carbon dioxide emission (and also manufacturing value added) of a country over the world total in every year from 1960 to 2020. 

In [29]:
# Calculate % share of CO2 emission by country by year
data2.loc[data2['Year'] == 1960, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13237,'CO2 Emission']*100
data2.loc[data2['Year'] == 1961, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13238,'CO2 Emission']*100
data2.loc[data2['Year'] == 1962, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13239,'CO2 Emission']*100
data2.loc[data2['Year'] == 1963, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13240,'CO2 Emission']*100
data2.loc[data2['Year'] == 1964, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13241,'CO2 Emission']*100
data2.loc[data2['Year'] == 1965, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13242,'CO2 Emission']*100
data2.loc[data2['Year'] == 1966, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13243,'CO2 Emission']*100
data2.loc[data2['Year'] == 1967, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13244,'CO2 Emission']*100
data2.loc[data2['Year'] == 1968, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13245,'CO2 Emission']*100
data2.loc[data2['Year'] == 1969, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13246,'CO2 Emission']*100
data2.loc[data2['Year'] == 1970, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13247,'CO2 Emission']*100
data2.loc[data2['Year'] == 1971, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13248,'CO2 Emission']*100
data2.loc[data2['Year'] == 1972, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13249,'CO2 Emission']*100
data2.loc[data2['Year'] == 1973, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13250,'CO2 Emission']*100
data2.loc[data2['Year'] == 1974, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13251,'CO2 Emission']*100
data2.loc[data2['Year'] == 1975, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13252,'CO2 Emission']*100
data2.loc[data2['Year'] == 1976, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13253,'CO2 Emission']*100
data2.loc[data2['Year'] == 1977, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13254,'CO2 Emission']*100
data2.loc[data2['Year'] == 1978, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13255,'CO2 Emission']*100
data2.loc[data2['Year'] == 1979, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13256,'CO2 Emission']*100
data2.loc[data2['Year'] == 1980, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13257,'CO2 Emission']*100
data2.loc[data2['Year'] == 1981, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13258,'CO2 Emission']*100
data2.loc[data2['Year'] == 1982, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13259,'CO2 Emission']*100
data2.loc[data2['Year'] == 1983, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13260,'CO2 Emission']*100
data2.loc[data2['Year'] == 1984, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13261,'CO2 Emission']*100
data2.loc[data2['Year'] == 1985, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13262,'CO2 Emission']*100
data2.loc[data2['Year'] == 1986, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13263,'CO2 Emission']*100
data2.loc[data2['Year'] == 1987, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13264,'CO2 Emission']*100
data2.loc[data2['Year'] == 1988, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13265,'CO2 Emission']*100
data2.loc[data2['Year'] == 1989, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13266,'CO2 Emission']*100
data2.loc[data2['Year'] == 1990, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13267,'CO2 Emission']*100
data2.loc[data2['Year'] == 1991, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13268,'CO2 Emission']*100
data2.loc[data2['Year'] == 1992, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13269,'CO2 Emission']*100
data2.loc[data2['Year'] == 1993, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13270,'CO2 Emission']*100
data2.loc[data2['Year'] == 1994, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13271,'CO2 Emission']*100
data2.loc[data2['Year'] == 1995, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13272,'CO2 Emission']*100
data2.loc[data2['Year'] == 1996, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13273,'CO2 Emission']*100
data2.loc[data2['Year'] == 1997, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13274,'CO2 Emission']*100
data2.loc[data2['Year'] == 1998, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13275,'CO2 Emission']*100
data2.loc[data2['Year'] == 1999, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13276,'CO2 Emission']*100
data2.loc[data2['Year'] == 2000, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13277,'CO2 Emission']*100
data2.loc[data2['Year'] == 2001, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13278,'CO2 Emission']*100
data2.loc[data2['Year'] == 2002, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13279,'CO2 Emission']*100
data2.loc[data2['Year'] == 2003, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13280,'CO2 Emission']*100
data2.loc[data2['Year'] == 2004, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13281,'CO2 Emission']*100
data2.loc[data2['Year'] == 2005, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13282,'CO2 Emission']*100
data2.loc[data2['Year'] == 2006, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13283,'CO2 Emission']*100
data2.loc[data2['Year'] == 2007, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13284,'CO2 Emission']*100
data2.loc[data2['Year'] == 2008, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13285,'CO2 Emission']*100
data2.loc[data2['Year'] == 2009, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13286,'CO2 Emission']*100
data2.loc[data2['Year'] == 2010, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13287,'CO2 Emission']*100
data2.loc[data2['Year'] == 2011, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13288,'CO2 Emission']*100
data2.loc[data2['Year'] == 2012, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13289,'CO2 Emission']*100
data2.loc[data2['Year'] == 2013, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13290,'CO2 Emission']*100
data2.loc[data2['Year'] == 2014, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13291,'CO2 Emission']*100
data2.loc[data2['Year'] == 2015, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13292,'CO2 Emission']*100
data2.loc[data2['Year'] == 2016, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13293,'CO2 Emission']*100
data2.loc[data2['Year'] == 2017, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13294,'CO2 Emission']*100
data2.loc[data2['Year'] == 2018, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13295,'CO2 Emission']*100
data2.loc[data2['Year'] == 2019, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13296,'CO2 Emission']*100
data2.loc[data2['Year'] == 2020, 'CO2 Country Share %'] = data2['CO2 Emission']/data2.loc[13297,'CO2 Emission']*100

In [30]:
# Calculate % share of manufacturing by country by year
data2.loc[data2['Year'] == 1960, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13237,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1961, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13238,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1962, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13239,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1963, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13240,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1964, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13241,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1965, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13242,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1966, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13243,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1967, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13244,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1968, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13245,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1969, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13246,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1970, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13247,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1971, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13248,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1972, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13249,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1973, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13250,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1974, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13251,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1975, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13252,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1976, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13253,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1977, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13254,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1978, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13255,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1979, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13256,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1980, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13257,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1981, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13258,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1982, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13259,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1983, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13260,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1984, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13261,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1985, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13262,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1986, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13263,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1987, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13264,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1988, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13265,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1989, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13266,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1990, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13267,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1991, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13268,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1992, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13269,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1993, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13270,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1994, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13271,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1995, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13272,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1996, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13273,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1997, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13274,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1998, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13275,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 1999, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13276,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 2000, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13277,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 2001, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13278,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 2002, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13279,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 2003, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13280,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 2004, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13281,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 2005, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13282,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 2006, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13283,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 2007, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13284,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 2008, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13285,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 2009, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13286,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 2010, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13287,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 2011, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13288,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 2012, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13289,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 2013, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13290,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 2014, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13291,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 2015, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13292,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 2016, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13293,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 2017, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13294,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 2018, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13295,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 2019, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13296,'Manufacturing GDP']*100
data2.loc[data2['Year'] == 2020, 'Manufacturing Country Share %'] = data2['Manufacturing GDP']/data2.loc[13297,'Manufacturing GDP']*100

### (C) Energy consumption and prevalence of renewable energy sources

Energy consumption and prevalence of renewable energy sources are two other factors which will have impact of carbon dioxide emission.  The data is mainly extracted from the energy balances compiled by International Energy Agency (IEA).
https://www.iea.org/data-and-statistics/data-product/world-energy-balances-highlights

#### (C1) Primary energy consumption, all sources

Primary energy consumption measures the total energy demand of a country. It covers consumption of the energy sector itself, losses during transformation (for example, from oil or gas into electricity) and distribution of energy, and the final consumption by end users.

The figures from IEA are expressed in a unit called 'petajoules'.

#### (C2) Primary energy consumption, renewables and waste

Renewables and waste comprises hydro, geothermal, solar, wind and tide/wave/ocean energy and the use of these energy forms for electricity and heat generation, as well as solid biofuels, liquid biofuels, biogases, industrial waste and municipal waste.

#### (C3) Electricity generation, all sources

Refers to electricity genereated by power plants and is expressed in gigawatt-hours.

#### (C4) Electricity generation, renewable sources

Refers to electricity generated by power plants using renewable sources and is expressed in gigawatt-hours.

In [31]:
# Get the path to raw data files
path_to_data = r'data/raw/complex'
path = str(pathlib.Path.cwd().parent.joinpath(path_to_data))+r"/"

# Read Energy Balance from IEA (energy and renewable consumption in petajoule (PJ), electricity and renewable production in gigawatt-hour (GWh))
xls = pd.ExcelFile(path+'World Energy Balances Highlights 2021.xlsx')
energy_balance = pd.read_excel(xls, 'TimeSeries_1971-2020', skiprows=[0])

# col_exclude = ['NoCountry','NoProduct','NoFlow','Unnamed: 56','Unnamed: 57','Unnamed: 58','Unnamed: 59','Unnamed: 60',
#                'Unnamed: 61','Unnamed: 62','Unnamed: 63','Unnamed: 64']

col_exclude = ['NoCountry', 'NoProduct', 'NoFlow']
energy_balance = energy_balance.drop(columns = col_exclude)


energy_balance=energy_balance.rename(columns={'2020 Provisional': 2020})
energy_balance=energy_balance.replace('..',np.nan)
energy_consumption=energy_balance[(energy_balance['Product']=='Total')&(energy_balance['Flow']=='Total energy supply (PJ)')]
energy_consumption.drop(['Product','Flow'],axis=1,inplace=True)

renewable_consumption=energy_balance[(energy_balance['Product']=='Renewables and waste')&(energy_balance['Flow']=='Total energy supply (PJ)')]
renewable_consumption.drop(['Product','Flow'],axis=1,inplace=True)
coal_consumption=energy_balance[(energy_balance['Product']=='Coal, peat and oil shale')&(energy_balance['Flow']=='Total energy supply (PJ)')]
coal_consumption.drop(['Product','Flow'],axis=1,inplace=True)
oil_consumption=energy_balance[(energy_balance['Product']=='Crude, NGL and feedstocks')&(energy_balance['Flow']=='Total energy supply (PJ)')]
oil_consumption.drop(['Product','Flow'],axis=1,inplace=True)
electricity_production=energy_balance[(energy_balance['Product']=='Total')&(energy_balance['Flow']=='Electricity output (GWh)')]
electricity_production.drop(['Product','Flow'],axis=1,inplace=True)
renewable_production=energy_balance[(energy_balance['Product']=='Renewable sources')&(energy_balance['Flow']=='Electricity output (GWh)')]
renewable_production.drop(['Product','Flow'],axis=1,inplace=True)

not_use=['OECD Total','Non-OECD Total','Africa','Non-OECD Americas','Non-OECD Asia (including China)','Non-OECD Europe and Eurasia',
         'Middle East','IEA Total','IEA and Accession/Association countries']
energy_consumption = energy_consumption[~energy_consumption['Country'].isin(not_use)]
renewable_consumption = renewable_consumption[~renewable_consumption['Country'].isin(not_use)]
coal_consumption = coal_consumption[~coal_consumption['Country'].isin(not_use)]
oil_consumption = oil_consumption[~oil_consumption['Country'].isin(not_use)]
electricity_production = electricity_production[~electricity_production['Country'].isin(not_use)]
renewable_production = renewable_production[~renewable_production['Country'].isin(not_use)]

index_number=list(range(46))
energy_consumption.index=index_number
renewable_consumption.index=index_number
coal_consumption.index=index_number
oil_consumption.index=index_number
electricity_production.index=index_number
renewable_production.index=index_number

code=['AUS','AUT','BEL','CAN','CHL','COL','CZE','DNK','EST','FIN','FRA','DEU','GRC','HUN','ISL','IRL','ISR','ITA',
      'JPN','KOR','LVA','LTU','LUX','MEX','NLD','NZL','NOR','POL','PRT','SVK','SVN','ESP','SWE','CHE','TUR','GBR',
      'USA','BRA','CHN','IND','IDN','MAR','SGP','ZAF','THA','WLD']
energy_consumption['Country Code']=code
renewable_consumption['Country Code']=code
coal_consumption['Country Code']=code
oil_consumption['Country Code']=code
electricity_production['Country Code']=code
renewable_production['Country Code']=code

Convert the datasets into long format and merge with the datasets in parts (A) and (B)

In [32]:
col_year_list3 = list(energy_consumption.columns.values)

print(col_year_list3)
print(energy_consumption)

col_year_list3 = col_year_list3[1:-1]
for i in col_year_list3:
    energy_consumption[i] = energy_consumption[i].astype(float)
energy_consumption=pd.melt(energy_consumption, id_vars=['Country','Country Code'], value_vars=col_year_list3) 
energy_consumption.rename(columns={'variable':'Year','value':'Primary Energy Consumption','Country':'Country Name'}, inplace=True)
energy_consumption['Year'] = energy_consumption['Year'].astype(int)
energy_consumption['Country Name'] = energy_consumption['Country Name'].str.strip() #strip leading blanks
energy_consumption.drop(['Country Name'],axis=1,inplace=True)

for i in col_year_list3:
    renewable_consumption[i] = renewable_consumption[i].astype(float)
renewable_consumption=pd.melt(renewable_consumption, id_vars=['Country','Country Code'], value_vars=col_year_list3) 
renewable_consumption.rename(columns={'variable':'Year','value':'Renewable Energy Consumption','Country':'Country Name'}, inplace=True)
renewable_consumption['Year'] = renewable_consumption['Year'].astype(int)
renewable_consumption['Country Name'] = renewable_consumption['Country Name'].str.strip() #strip leading blanks
renewable_consumption.drop(['Country Name'],axis=1,inplace=True)

for i in col_year_list3:
    coal_consumption[i] = coal_consumption[i].astype(float)
coal_consumption=pd.melt(coal_consumption, id_vars=['Country','Country Code'], value_vars=col_year_list3) 
coal_consumption.rename(columns={'variable':'Year','value':'Coal Consumption','Country':'Country Name'}, inplace=True)
coal_consumption['Year'] = coal_consumption['Year'].astype(int)
coal_consumption['Country Name'] = coal_consumption['Country Name'].str.strip() #strip leading blanks
coal_consumption.drop(['Country Name'],axis=1,inplace=True)

for i in col_year_list3:
    oil_consumption[i] = oil_consumption[i].astype(float)
oil_consumption=pd.melt(oil_consumption, id_vars=['Country','Country Code'], value_vars=col_year_list3) 
oil_consumption.rename(columns={'variable':'Year','value':'Oil Consumption','Country':'Country Name'}, inplace=True)
oil_consumption['Year'] = oil_consumption['Year'].astype(int)
oil_consumption['Country Name'] = oil_consumption['Country Name'].str.strip() #strip leading blanks
oil_consumption.drop(['Country Name'],axis=1,inplace=True)

for i in col_year_list3:
    electricity_production[i] = electricity_production[i].astype(float)
electricity_production=pd.melt(electricity_production, id_vars=['Country','Country Code'], value_vars=col_year_list3) 
electricity_production.rename(columns={'variable':'Year','value':'Total Electricity Production','Country':'Country Name'}, inplace=True)
electricity_production['Year'] = electricity_production['Year'].astype(int)
electricity_production['Country Name'] = electricity_production['Country Name'].str.strip() #strip leading blanks
electricity_production.drop(['Country Name'],axis=1,inplace=True)

for i in col_year_list3:
    renewable_production[i] = renewable_production[i].astype(float)
renewable_production=pd.melt(renewable_production, id_vars=['Country','Country Code'], value_vars=col_year_list3) 
renewable_production.rename(columns={'variable':'Year','value':'Electricity Production from Renewable','Country':'Country Name'}, inplace=True)
renewable_production['Year'] = renewable_production['Year'].astype(int)
renewable_production['Country Name'] = renewable_production['Country Name'].str.strip() #strip leading blanks
renewable_production.drop(['Country Name'],axis=1,inplace=True)

['Country', 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 'Country Code']
                       Country           1971           1972           1973  \
0                    Australia    2161.010837    2226.254215    2388.879855   
1                      Austria     787.781807     829.899570     899.186601   
2                      Belgium    1660.521845    1819.103611    1925.534470   
3                       Canada    5918.473275    6385.021633    6671.850798   
4                        Chile     364.432616     370.059609     355.911628   
5                     Colombia     579.989077     580.515290     583.929278   
6               Czech Republic    1900.156014    1906.129699    1890.734040   
7                      Denmark     774.7

In [33]:
# Merge the merged dataset with Energy Balance from IEA
data2=pd.merge(data2,energy_consumption,how='left',on=['Country Code','Year'])
data2=pd.merge(data2,renewable_consumption,how='left',on=['Country Code','Year'])
data2=pd.merge(data2,coal_consumption,how='left',on=['Country Code','Year'])
data2=pd.merge(data2,oil_consumption,how='left',on=['Country Code','Year'])
data2=pd.merge(data2,electricity_production,how='left',on=['Country Code','Year'])
data2=pd.merge(data2,renewable_production,how='left',on=['Country Code','Year'])

Derive some features in the merged dataset, which may be useful for our analysis:
* Primary energy consumption per capita (in kg of oil equivalent) = (Primary energy consumption * 23884.58966275 * 1000)/Total population

* Share of renewable sources in primary energy consumption (percent) = (Primary energy consumption from renewable sources / Total primary energy consumption) * 100

* Share of renewable sources in electricity generation (percent) = (Electricity production from renewable sources / Total electricity production) * 100

Note:
1 petajoule = 23884.58966275 ton of oil equivalent = 23884.58966275 * 1000 kg of oil equivalent

In [34]:
# Calculate primary energy consumption per capita (kg of oil equivalent per person)
data2['Primary Energy Consumption per capita']=(data2['Primary Energy Consumption']*23884.58966275*1000)/data2['Population'] # in kg of oil equivalent per person
data2['Renewable Energy Consumption Share']=(data2['Renewable Energy Consumption']/data2['Primary Energy Consumption'])*100
data2['Fossil Energy Consumption Share']=(data2['Coal Consumption']+data2['Oil Consumption'])/data2['Primary Energy Consumption']*100
data2['Renewable Electricity Production Share']=(data2['Electricity Production from Renewable']/data2['Total Electricity Production'])*100

Free energy balance data from IEA only cover OECD countries and some major non-OECD countries like China.  To achieve a more comprehensive country coverage, the energy use per capita figures, which can be downloaded from the World Bank data platform (https://data.worldbank.org/indicator/EG.USE.PCAP.KG.OE) and is also sourced from IEA, are incorporated into the feature 'Primary energy consumption per capita'.

With these additional primary energy consumption per capita data, the additional primary energy consumption data can also be derived by multiplying the per capita data by the population figure, then convert the unit from kg of oil equivalent to petajoules.

Note: 1 kg of oil equivalent = 0.000041868/1000 petajoules

In [35]:
# Use 'Energy use per capita' from World Bank to fill in as much missing data as possible in 'Primary Energy Consumption per capita'
# For the column 'primary energy consumption per capita', replace nan by value in 'energy use per capita'
data2['Primary Energy Consumption per capita'] = np.where(~data2['Primary Energy Consumption per capita'].isnull(),data2['Primary Energy Consumption per capita'],data2['Energy Use per capita'])
data2.drop(['Energy Use per capita'],axis=1,inplace=True)

# For the column 'primary energy consumption' (in PJ), replace nan by value calculated from 'primary energy consumption per capita'
data2['Primary Energy Consumption'] = np.where(~data2['Primary Energy Consumption'].isnull(),data2['Primary Energy Consumption'],data2['Primary Energy Consumption per capita']*data2['Population']*0.000041868/1000)

Since the energy use per capita series from World Bank only up to 2014, year-on-year change in primary energy consumption obtained from 'British Petroleum Statistical Review of World Energy' (https://www.bp.com/en/global/corporate/energy-economics/statistical-review-of-world-energy.html) is also incorporated where appropriate to derive the primary energy consumption and primary energy consumption per capita figures from 2015 to 2020. 

In [36]:
# Estimate primary energy consumption data using year-on-year change in BP Statistical Review (in PJ)
# China
data2.loc[13056,'Primary Energy Consumption']=data2.loc[12839,'Primary Energy Consumption']*(1+0.021) #2020

# Russia (Assume 80% of USSR data belongs to Russia based on 1985 situation)
data2.loc[1250,'Primary Energy Consumption']=24.8478116832472*0.8*1000 #1965
data2.loc[1467,'Primary Energy Consumption']=26.4226954380792*0.8*1000 #1966
data2.loc[1684,'Primary Energy Consumption']=27.7739292062979*0.8*1000 #1967
data2.loc[1901,'Primary Energy Consumption']=28.8050192841883*0.8*1000 #1968
data2.loc[2118,'Primary Energy Consumption']=30.0728112068223*0.8*1000 #1969
data2.loc[2335,'Primary Energy Consumption']=31.634512141811*0.8*1000 #1970
data2.loc[2552,'Primary Energy Consumption']=33.1899588753875*0.8*1000 #1971
data2.loc[2769,'Primary Energy Consumption']=34.8990186312964*0.8*1000 #1972
data2.loc[2986,'Primary Energy Consumption']=36.6740657474514*0.8*1000 #1973
data2.loc[3203,'Primary Energy Consumption']=38.6988392951834*0.8*1000 #1974
data2.loc[3420,'Primary Energy Consumption']=40.6154751507423*0.8*1000 #1975
data2.loc[3637,'Primary Energy Consumption']=42.1943824215537*0.8*1000 #1976
data2.loc[3854,'Primary Energy Consumption']=44.0818489328113*0.8*1000 #1977
data2.loc[4071,'Primary Energy Consumption']=45.9804353150276*0.8*1000 #1978
data2.loc[4288,'Primary Energy Consumption']=47.2396569295646*0.8*1000 #1979
data2.loc[4505,'Primary Energy Consumption']=48.2912900352032*0.8*1000 #1980
data2.loc[4722,'Primary Energy Consumption']=49.3185017304752*0.8*1000 #1981
data2.loc[4939,'Primary Energy Consumption']=50.5546609494802*0.8*1000 #1982
data2.loc[5156,'Primary Energy Consumption']=51.6172895305755*0.8*1000 #1983
data2.loc[5373,'Primary Energy Consumption']=53.5518568466529*0.8*1000 #1984
data2.loc[5590,'Primary Energy Consumption']=34.2740920017715*1000 #1985
data2.loc[5807,'Primary Energy Consumption']=34.8793921166512*1000 #1986
data2.loc[6024,'Primary Energy Consumption']=36.0217743703685*1000 #1987
data2.loc[6241,'Primary Energy Consumption']=36.6522064745222*1000 #1988
data2.loc[6458,'Primary Energy Consumption']=36.7886170072215*1000 #1989
data2.loc[12100,'Primary Energy Consumption']=data2.loc[11883,'Primary Energy Consumption']*0.983 #2015
data2.loc[12317,'Primary Energy Consumption']=data2.loc[12100,'Primary Energy Consumption']*1.021 #2016
data2.loc[12534,'Primary Energy Consumption']=data2.loc[12317,'Primary Energy Consumption']*1.006 #2017
data2.loc[12751,'Primary Energy Consumption']=data2.loc[12534,'Primary Energy Consumption']*1.038 #2018
data2.loc[12968,'Primary Energy Consumption']=data2.loc[12751,'Primary Energy Consumption']*0.993 #2019
data2.loc[13185,'Primary Energy Consumption']=data2.loc[12968,'Primary Energy Consumption']*0.947 #2020

# Saudi Arabia
data2.loc[data2.index[(data2['Country Code']=='SAU')&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='SAU')&(data2['Year']==2014)][0],'Primary Energy Consumption']*1.031 #2015
data2.loc[data2.index[(data2['Country Code']=='SAU')&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='SAU')&(data2['Year']==2015)][0],'Primary Energy Consumption']*1.013 #2016
data2.loc[data2.index[(data2['Country Code']=='SAU')&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='SAU')&(data2['Year']==2016)][0],'Primary Energy Consumption']*0.998 #2017
data2.loc[data2.index[(data2['Country Code']=='SAU')&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='SAU')&(data2['Year']==2017)][0],'Primary Energy Consumption']*0.974 #2018
data2.loc[data2.index[(data2['Country Code']=='SAU')&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='SAU')&(data2['Year']==2018)][0],'Primary Energy Consumption']*1.003 #2019
data2.loc[data2.index[(data2['Country Code']=='SAU')&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='SAU')&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.989 #2020

# India
data2.loc[13109,'Primary Energy Consumption']=data2.loc[12892,'Primary Energy Consumption']*0.944 #2020

# Indonesia
data2.loc[13107,'Primary Energy Consumption']=data2.loc[12890,'Primary Energy Consumption']*0.882 #2020

# Singapore
data2.loc[13190,'Primary Energy Consumption']=data2.loc[12973,'Primary Energy Consumption']*0.968 #2020

# South Africa
data2.loc[13234,'Primary Energy Consumption']=data2.loc[13017,'Primary Energy Consumption']*0.941 #2020

# Iran
data2.loc[data2.index[(data2['Country Code']=='IRN')&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='IRN')&(data2['Year']==2014)][0],'Primary Energy Consumption']*0.995 #2015
data2.loc[data2.index[(data2['Country Code']=='IRN')&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='IRN')&(data2['Year']==2015)][0],'Primary Energy Consumption']*1.046 #2016
data2.loc[data2.index[(data2['Country Code']=='IRN')&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='IRN')&(data2['Year']==2016)][0],'Primary Energy Consumption']*1.038 #2017
data2.loc[data2.index[(data2['Country Code']=='IRN')&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='IRN')&(data2['Year']==2017)][0],'Primary Energy Consumption']*1.058 #2018
data2.loc[data2.index[(data2['Country Code']=='IRN')&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='IRN')&(data2['Year']==2018)][0],'Primary Energy Consumption']*1.049 #2019
data2.loc[data2.index[(data2['Country Code']=='IRN')&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='IRN')&(data2['Year']==2019)][0],'Primary Energy Consumption']*1.005 #2020

# Kazakhstan
data2.loc[data2.index[(data2['Country Code']=='KAZ')&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='KAZ')&(data2['Year']==2014)][0],'Primary Energy Consumption']*1.006 #2015
data2.loc[data2.index[(data2['Country Code']=='KAZ')&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='KAZ')&(data2['Year']==2015)][0],'Primary Energy Consumption']*0.994 #2016
data2.loc[data2.index[(data2['Country Code']=='KAZ')&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='KAZ')&(data2['Year']==2016)][0],'Primary Energy Consumption']*1.078 #2017
data2.loc[data2.index[(data2['Country Code']=='KAZ')&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='KAZ')&(data2['Year']==2017)][0],'Primary Energy Consumption']*1.102 #2018
data2.loc[data2.index[(data2['Country Code']=='KAZ')&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='KAZ')&(data2['Year']==2018)][0],'Primary Energy Consumption']*0.972 #2019
data2.loc[data2.index[(data2['Country Code']=='KAZ')&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='KAZ')&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.986 #2020

# Argentina
data2.loc[data2.index[(data2['Country Code']=='ARG')&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='ARG')&(data2['Year']==2014)][0],'Primary Energy Consumption']*1.019 #2015
data2.loc[data2.index[(data2['Country Code']=='ARG')&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='ARG')&(data2['Year']==2015)][0],'Primary Energy Consumption']*0.993 #2016
data2.loc[data2.index[(data2['Country Code']=='ARG')&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='ARG')&(data2['Year']==2016)][0],'Primary Energy Consumption']*1.002 #2017
data2.loc[data2.index[(data2['Country Code']=='ARG')&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='ARG')&(data2['Year']==2017)][0],'Primary Energy Consumption']*0.986 #2018
data2.loc[data2.index[(data2['Country Code']=='ARG')&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='ARG')&(data2['Year']==2018)][0],'Primary Energy Consumption']*0.942 #2019
data2.loc[data2.index[(data2['Country Code']=='ARG')&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='ARG')&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.931 #2020

# Ecuador
data2.loc[data2.index[(data2['Country Code']=='ECU')&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='ECU')&(data2['Year']==2014)][0],'Primary Energy Consumption']*0.996 #2015
data2.loc[data2.index[(data2['Country Code']=='ECU')&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='ECU')&(data2['Year']==2015)][0],'Primary Energy Consumption']*0.994 #2016
data2.loc[data2.index[(data2['Country Code']=='ECU')&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='ECU')&(data2['Year']==2016)][0],'Primary Energy Consumption']*1.035 #2017
data2.loc[data2.index[(data2['Country Code']=='ECU')&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='ECU')&(data2['Year']==2017)][0],'Primary Energy Consumption']*1.057 #2018
data2.loc[data2.index[(data2['Country Code']=='ECU')&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='ECU')&(data2['Year']==2018)][0],'Primary Energy Consumption']*1.025 #2019
data2.loc[data2.index[(data2['Country Code']=='ECU')&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='ECU')&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.867 #2020

# Peru
data2.loc[data2.index[(data2['Country Code']=='PER')&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='PER')&(data2['Year']==2014)][0],'Primary Energy Consumption']*1.048 #2015
data2.loc[data2.index[(data2['Country Code']=='PER')&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='PER')&(data2['Year']==2015)][0],'Primary Energy Consumption']*1.069 #2016
data2.loc[data2.index[(data2['Country Code']=='PER')&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='PER')&(data2['Year']==2016)][0],'Primary Energy Consumption']*1.025 #2017
data2.loc[data2.index[(data2['Country Code']=='PER')&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='PER')&(data2['Year']==2017)][0],'Primary Energy Consumption']*1.054 #2018
data2.loc[data2.index[(data2['Country Code']=='PER')&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='PER')&(data2['Year']==2018)][0],'Primary Energy Consumption']*1.010 #2019
data2.loc[data2.index[(data2['Country Code']=='PER')&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='PER')&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.841 #2020

# Trindad
data2.loc[data2.index[(data2['Country Code']=='TTO')&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='TTO')&(data2['Year']==2014)][0],'Primary Energy Consumption']*0.970 #2015
data2.loc[data2.index[(data2['Country Code']=='TTO')&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='TTO')&(data2['Year']==2015)][0],'Primary Energy Consumption']*0.887 #2016
data2.loc[data2.index[(data2['Country Code']=='TTO')&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='TTO')&(data2['Year']==2016)][0],'Primary Energy Consumption']*1.061 #2017
data2.loc[data2.index[(data2['Country Code']=='TTO')&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='TTO')&(data2['Year']==2017)][0],'Primary Energy Consumption']*0.947 #2018
data2.loc[data2.index[(data2['Country Code']=='TTO')&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='TTO')&(data2['Year']==2018)][0],'Primary Energy Consumption']*0.997 #2019
data2.loc[data2.index[(data2['Country Code']=='TTO')&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='TTO')&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.878 #2020

# Venezuela
data2.loc[data2.index[(data2['Country Code']=='VEN')&(data2['Year']==2014)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='VEN')&(data2['Year']==2013)][0],'Primary Energy Consumption']*0.951 #2014
data2.loc[data2.index[(data2['Country Code']=='VEN')&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='VEN')&(data2['Year']==2014)][0],'Primary Energy Consumption']*0.988 #2015
data2.loc[data2.index[(data2['Country Code']=='VEN')&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='VEN')&(data2['Year']==2015)][0],'Primary Energy Consumption']*0.871 #2016
data2.loc[data2.index[(data2['Country Code']=='VEN')&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='VEN')&(data2['Year']==2016)][0],'Primary Energy Consumption']*0.978 #2017
data2.loc[data2.index[(data2['Country Code']=='VEN')&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='VEN')&(data2['Year']==2017)][0],'Primary Energy Consumption']*0.846 #2018
data2.loc[data2.index[(data2['Country Code']=='VEN')&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='VEN')&(data2['Year']==2018)][0],'Primary Energy Consumption']*0.788 #2019
data2.loc[data2.index[(data2['Country Code']=='VEN')&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='VEN')&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.739 #2020

# Bulgaria
data2.loc[data2.index[(data2['Country Code']=='BGR')&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='BGR')&(data2['Year']==2014)][0],'Primary Energy Consumption']*1.063 #2015
data2.loc[data2.index[(data2['Country Code']=='BGR')&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='BGR')&(data2['Year']==2015)][0],'Primary Energy Consumption']*0.947 #2016
data2.loc[data2.index[(data2['Country Code']=='BGR')&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='BGR')&(data2['Year']==2016)][0],'Primary Energy Consumption']*1.023 #2017
data2.loc[data2.index[(data2['Country Code']=='BGR')&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='BGR')&(data2['Year']==2017)][0],'Primary Energy Consumption']*0.998 #2018
data2.loc[data2.index[(data2['Country Code']=='BGR')&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='BGR')&(data2['Year']==2018)][0],'Primary Energy Consumption']*0.966 #2019
data2.loc[data2.index[(data2['Country Code']=='BGR')&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='BGR')&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.927 #2020

# Crotia
data2.loc[data2.index[(data2['Country Code']=='HRV')&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='HRV')&(data2['Year']==2014)][0],'Primary Energy Consumption']*0.951 #2015
data2.loc[data2.index[(data2['Country Code']=='HRV')&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='HRV')&(data2['Year']==2015)][0],'Primary Energy Consumption']*1.039 #2016
data2.loc[data2.index[(data2['Country Code']=='HRV')&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='HRV')&(data2['Year']==2016)][0],'Primary Energy Consumption']*1.002 #2017
data2.loc[data2.index[(data2['Country Code']=='HRV')&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='HRV')&(data2['Year']==2017)][0],'Primary Energy Consumption']*1.031 #2018
data2.loc[data2.index[(data2['Country Code']=='HRV')&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='HRV')&(data2['Year']==2018)][0],'Primary Energy Consumption']*0.977 #2019
data2.loc[data2.index[(data2['Country Code']=='HRV')&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='HRV')&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.965 #2020

# Romania
data2.loc[data2.index[(data2['Country Code']=='ROU')&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='ROU')&(data2['Year']==2014)][0],'Primary Energy Consumption']*1.005 #2015
data2.loc[data2.index[(data2['Country Code']=='ROU')&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='ROU')&(data2['Year']==2015)][0],'Primary Energy Consumption']*1 #2016
data2.loc[data2.index[(data2['Country Code']=='ROU')&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='ROU')&(data2['Year']==2016)][0],'Primary Energy Consumption']*1.018 #2017
data2.loc[data2.index[(data2['Country Code']=='ROU')&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='ROU')&(data2['Year']==2017)][0],'Primary Energy Consumption']*1.019 #2018
data2.loc[data2.index[(data2['Country Code']=='ROU')&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='ROU')&(data2['Year']==2018)][0],'Primary Energy Consumption']*0.977 #2019
data2.loc[data2.index[(data2['Country Code']=='ROU')&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='ROU')&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.967 #2020

# Uzbekistan
data2.loc[data2.index[(data2['Country Code']=='UZB')&(data2['Year']==2014)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='UZB')&(data2['Year']==2013)][0],'Primary Energy Consumption']*1.032 #2014
data2.loc[data2.index[(data2['Country Code']=='UZB')&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='UZB')&(data2['Year']==2014)][0],'Primary Energy Consumption']*0.954 #2015
data2.loc[data2.index[(data2['Country Code']=='UZB')&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='UZB')&(data2['Year']==2015)][0],'Primary Energy Consumption']*0.958 #2016
data2.loc[data2.index[(data2['Country Code']=='UZB')&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='UZB')&(data2['Year']==2016)][0],'Primary Energy Consumption']*1.003 #2017
data2.loc[data2.index[(data2['Country Code']=='UZB')&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='UZB')&(data2['Year']==2017)][0],'Primary Energy Consumption']*1.035 #2018
data2.loc[data2.index[(data2['Country Code']=='UZB')&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='UZB')&(data2['Year']==2018)][0],'Primary Energy Consumption']*1.002 #2019
data2.loc[data2.index[(data2['Country Code']=='UZB')&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='UZB')&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.975 #2020

# Ukraine
data2.loc[data2.index[(data2['Country Code']=='UKR')&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='UKR')&(data2['Year']==2014)][0],'Primary Energy Consumption']*0.829 #2015
data2.loc[data2.index[(data2['Country Code']=='UKR')&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='UKR')&(data2['Year']==2015)][0],'Primary Energy Consumption']*1.046 #2016
data2.loc[data2.index[(data2['Country Code']=='UKR')&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='UKR')&(data2['Year']==2016)][0],'Primary Energy Consumption']*0.930 #2017
data2.loc[data2.index[(data2['Country Code']=='UKR')&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='UKR')&(data2['Year']==2017)][0],'Primary Energy Consumption']*1.036 #2018
data2.loc[data2.index[(data2['Country Code']=='UKR')&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='UKR')&(data2['Year']==2018)][0],'Primary Energy Consumption']*0.945 #2019
data2.loc[data2.index[(data2['Country Code']=='UKR')&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='UKR')&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.969 #2020

# Turkmenistan
data2.loc[data2.index[(data2['Country Code']=='TKM')&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='TKM')&(data2['Year']==2014)][0],'Primary Energy Consumption']*1.197 #2015
data2.loc[data2.index[(data2['Country Code']=='TKM')&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='TKM')&(data2['Year']==2015)][0],'Primary Energy Consumption']*0.99 #2016
data2.loc[data2.index[(data2['Country Code']=='TKM')&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='TKM')&(data2['Year']==2016)][0],'Primary Energy Consumption']*0.99 #2017
data2.loc[data2.index[(data2['Country Code']=='TKM')&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='TKM')&(data2['Year']==2017)][0],'Primary Energy Consumption']*1.113 #2018
data2.loc[data2.index[(data2['Country Code']=='TKM')&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='TKM')&(data2['Year']==2018)][0],'Primary Energy Consumption']*1.092 #2019
data2.loc[data2.index[(data2['Country Code']=='TKM')&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='TKM')&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.984 #2020

# Iraq
data2.loc[data2.index[(data2['Country Code']=='IRQ')&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='IRQ')&(data2['Year']==2014)][0],'Primary Energy Consumption']*0.999 #2015
data2.loc[data2.index[(data2['Country Code']=='IRQ')&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='IRQ')&(data2['Year']==2015)][0],'Primary Energy Consumption']*1.147 #2016
data2.loc[data2.index[(data2['Country Code']=='IRQ')&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='IRQ')&(data2['Year']==2016)][0],'Primary Energy Consumption']*0.987 #2017
data2.loc[data2.index[(data2['Country Code']=='IRQ')&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='IRQ')&(data2['Year']==2017)][0],'Primary Energy Consumption']*1.048 #2018
data2.loc[data2.index[(data2['Country Code']=='IRQ')&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='IRQ')&(data2['Year']==2018)][0],'Primary Energy Consumption']*1.098 #2019
data2.loc[data2.index[(data2['Country Code']=='IRQ')&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='IRQ')&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.940 #2020

# Kuwait
data2.loc[data2.index[(data2['Country Code']=='KWT')&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='KWT')&(data2['Year']==2014)][0],'Primary Energy Consumption']*1.027 #2015
data2.loc[data2.index[(data2['Country Code']=='KWT')&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='KWT')&(data2['Year']==2015)][0],'Primary Energy Consumption']*1.0007 #2016
data2.loc[data2.index[(data2['Country Code']=='KWT')&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='KWT')&(data2['Year']==2016)][0],'Primary Energy Consumption']*1.019 #2017
data2.loc[data2.index[(data2['Country Code']=='KWT')&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='KWT')&(data2['Year']==2017)][0],'Primary Energy Consumption']*1.0007 #2018
data2.loc[data2.index[(data2['Country Code']=='KWT')&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='KWT')&(data2['Year']==2018)][0],'Primary Energy Consumption']*1.012 #2019
data2.loc[data2.index[(data2['Country Code']=='KWT')&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='KWT')&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.903 #2020

# Qatar
data2.loc[data2.index[(data2['Country Code']=='QAT')&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='QAT')&(data2['Year']==2014)][0],'Primary Energy Consumption']*1.14 #2015
data2.loc[data2.index[(data2['Country Code']=='QAT')&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='QAT')&(data2['Year']==2015)][0],'Primary Energy Consumption']*0.977 #2016
data2.loc[data2.index[(data2['Country Code']=='QAT')&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='QAT')&(data2['Year']==2016)][0],'Primary Energy Consumption']*0.971 #2017
data2.loc[data2.index[(data2['Country Code']=='QAT')&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='QAT')&(data2['Year']==2017)][0],'Primary Energy Consumption']*0.882 #2018
data2.loc[data2.index[(data2['Country Code']=='QAT')&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='QAT')&(data2['Year']==2018)][0],'Primary Energy Consumption']*1.082 #2019
data2.loc[data2.index[(data2['Country Code']=='QAT')&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='QAT')&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.889 #2020

# United Arab Emirates
data2.loc[data2.index[(data2['Country Code']=='ARE')&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='ARE')&(data2['Year']==2014)][0],'Primary Energy Consumption']*1.106 #2015
data2.loc[data2.index[(data2['Country Code']=='ARE')&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='ARE')&(data2['Year']==2015)][0],'Primary Energy Consumption']*1.04 #2016
data2.loc[data2.index[(data2['Country Code']=='ARE')&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='ARE')&(data2['Year']==2016)][0],'Primary Energy Consumption']*1.003 #2017
data2.loc[data2.index[(data2['Country Code']=='ARE')&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='ARE')&(data2['Year']==2017)][0],'Primary Energy Consumption']*0.986 #2018
data2.loc[data2.index[(data2['Country Code']=='ARE')&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='ARE')&(data2['Year']==2018)][0],'Primary Energy Consumption']*0.992 #2019
data2.loc[data2.index[(data2['Country Code']=='ARE')&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='ARE')&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.920 #2020

# Oman
data2.loc[data2.index[(data2['Country Code']=='OMN')&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='OMN')&(data2['Year']==2014)][0],'Primary Energy Consumption']*1.057 #2015
data2.loc[data2.index[(data2['Country Code']=='OMN')&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='OMN')&(data2['Year']==2015)][0],'Primary Energy Consumption']*1.002 #2016
data2.loc[data2.index[(data2['Country Code']=='OMN')&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='OMN')&(data2['Year']==2016)][0],'Primary Energy Consumption']*1.074 #2017
data2.loc[data2.index[(data2['Country Code']=='OMN')&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='OMN')&(data2['Year']==2017)][0],'Primary Energy Consumption']*1.054 #2018
data2.loc[data2.index[(data2['Country Code']=='OMN')&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='OMN')&(data2['Year']==2018)][0],'Primary Energy Consumption']*1.014 #2019
data2.loc[data2.index[(data2['Country Code']=='OMN')&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='OMN')&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.985 #2020

# Azerbaijan
data2.loc[data2.index[(data2['Country Code']=='AZE')&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='AZE')&(data2['Year']==2014)][0],'Primary Energy Consumption']*1.093 #2015
data2.loc[data2.index[(data2['Country Code']=='AZE')&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='AZE')&(data2['Year']==2015)][0],'Primary Energy Consumption']*0.99 #2016
data2.loc[data2.index[(data2['Country Code']=='AZE')&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='AZE')&(data2['Year']==2016)][0],'Primary Energy Consumption']*0.981 #2017
data2.loc[data2.index[(data2['Country Code']=='AZE')&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='AZE')&(data2['Year']==2017)][0],'Primary Energy Consumption']*1.029 #2018
data2.loc[data2.index[(data2['Country Code']=='AZE')&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='AZE')&(data2['Year']==2018)][0],'Primary Energy Consumption']*1.053 #2019
data2.loc[data2.index[(data2['Country Code']=='AZE')&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='AZE')&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.959 #2020

# Belarus
data2.loc[data2.index[(data2['Country Code']=='BLR')&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='BLR')&(data2['Year']==2014)][0],'Primary Energy Consumption']*0.909 #2015
data2.loc[data2.index[(data2['Country Code']=='BLR')&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='BLR')&(data2['Year']==2015)][0],'Primary Energy Consumption']*0.995 #2016
data2.loc[data2.index[(data2['Country Code']=='BLR')&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='BLR')&(data2['Year']==2016)][0],'Primary Energy Consumption']*1.02 #2017
data2.loc[data2.index[(data2['Country Code']=='BLR')&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='BLR')&(data2['Year']==2017)][0],'Primary Energy Consumption']*1.088 #2018
data2.loc[data2.index[(data2['Country Code']=='BLR')&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='BLR')&(data2['Year']==2018)][0],'Primary Energy Consumption']*0.985 #2019
data2.loc[data2.index[(data2['Country Code']=='BLR')&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='BLR')&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.932 #2020

# Morocco
data2.loc[data2.index[(data2['Country Code']=='MAR')&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='MAR')&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.931 #2020

# Algeria
data2.loc[data2.index[(data2['Country Code']=='DZA')&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='DZA')&(data2['Year']==2014)][0],'Primary Energy Consumption']*1.054 #2015
data2.loc[data2.index[(data2['Country Code']=='DZA')&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='DZA')&(data2['Year']==2015)][0],'Primary Energy Consumption']*0.999 #2016
data2.loc[data2.index[(data2['Country Code']=='DZA')&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='DZA')&(data2['Year']==2016)][0],'Primary Energy Consumption']*1.011 #2017
data2.loc[data2.index[(data2['Country Code']=='DZA')&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='DZA')&(data2['Year']==2017)][0],'Primary Energy Consumption']*1.076 #2018
data2.loc[data2.index[(data2['Country Code']=='DZA')&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='DZA')&(data2['Year']==2018)][0],'Primary Energy Consumption']*1.036 #2019
data2.loc[data2.index[(data2['Country Code']=='DZA')&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='DZA')&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.919 #2020

# Egypt
data2.loc[data2.index[(data2['Country Code']=='EGY')&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='EGY')&(data2['Year']==2014)][0],'Primary Energy Consumption']*1.022 #2015
data2.loc[data2.index[(data2['Country Code']=='EGY')&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='EGY')&(data2['Year']==2015)][0],'Primary Energy Consumption']*1.050 #2016
data2.loc[data2.index[(data2['Country Code']=='EGY')&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='EGY')&(data2['Year']==2016)][0],'Primary Energy Consumption']*1.028 #2017
data2.loc[data2.index[(data2['Country Code']=='EGY')&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='EGY')&(data2['Year']==2017)][0],'Primary Energy Consumption']*1.014 #2018
data2.loc[data2.index[(data2['Country Code']=='EGY')&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='EGY')&(data2['Year']==2018)][0],'Primary Energy Consumption']*1.002 #2019
data2.loc[data2.index[(data2['Country Code']=='EGY')&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='EGY')&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.944 #2020

# Bangladesh
data2.loc[data2.index[(data2['Country Code']=='BGD')&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='BGD')&(data2['Year']==2014)][0],'Primary Energy Consumption']*1.163 #2015
data2.loc[data2.index[(data2['Country Code']=='BGD')&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='BGD')&(data2['Year']==2015)][0],'Primary Energy Consumption']*1.017 #2016
data2.loc[data2.index[(data2['Country Code']=='BGD')&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='BGD')&(data2['Year']==2016)][0],'Primary Energy Consumption']*1.038 #2017
data2.loc[data2.index[(data2['Country Code']=='BGD')&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='BGD')&(data2['Year']==2017)][0],'Primary Energy Consumption']*1.061 #2018
data2.loc[data2.index[(data2['Country Code']=='BGD')&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='BGD')&(data2['Year']==2018)][0],'Primary Energy Consumption']*1.119 #2019
data2.loc[data2.index[(data2['Country Code']=='BGD')&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='BGD')&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.971 #2020

# Malaysia
data2.loc[data2.index[(data2['Country Code']=='MYS')&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='MYS')&(data2['Year']==2014)][0],'Primary Energy Consumption']*1.014 #2015
data2.loc[data2.index[(data2['Country Code']=='MYS')&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='MYS')&(data2['Year']==2015)][0],'Primary Energy Consumption']*1.053 #2016
data2.loc[data2.index[(data2['Country Code']=='MYS')&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='MYS')&(data2['Year']==2016)][0],'Primary Energy Consumption']*1.015 #2017
data2.loc[data2.index[(data2['Country Code']=='MYS')&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='MYS')&(data2['Year']==2017)][0],'Primary Energy Consumption']*1.014 #2018
data2.loc[data2.index[(data2['Country Code']=='MYS')&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='MYS')&(data2['Year']==2018)][0],'Primary Energy Consumption']*1.024 #2019
data2.loc[data2.index[(data2['Country Code']=='MYS')&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='MYS')&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.928 #2020

# Pakistan
data2.loc[data2.index[(data2['Country Code']=='PAK')&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='PAK')&(data2['Year']==2014)][0],'Primary Energy Consumption']*1.051 #2015
data2.loc[data2.index[(data2['Country Code']=='PAK')&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='PAK')&(data2['Year']==2015)][0],'Primary Energy Consumption']*1.091 #2016
data2.loc[data2.index[(data2['Country Code']=='PAK')&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='PAK')&(data2['Year']==2016)][0],'Primary Energy Consumption']*1.060 #2017
data2.loc[data2.index[(data2['Country Code']=='PAK')&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='PAK')&(data2['Year']==2017)][0],'Primary Energy Consumption']*1.032 #2018
data2.loc[data2.index[(data2['Country Code']=='PAK')&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='PAK')&(data2['Year']==2018)][0],'Primary Energy Consumption']*1.01 #2019
data2.loc[data2.index[(data2['Country Code']=='PAK')&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='PAK')&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.988 #2020

# The Philippines
data2.loc[data2.index[(data2['Country Code']=='PHL')&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='PHL')&(data2['Year']==2014)][0],'Primary Energy Consumption']*1.096 #2015
data2.loc[data2.index[(data2['Country Code']=='PHL')&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='PHL')&(data2['Year']==2015)][0],'Primary Energy Consumption']*1.092 #2016
data2.loc[data2.index[(data2['Country Code']=='PHL')&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='PHL')&(data2['Year']==2016)][0],'Primary Energy Consumption']*1.097 #2017
data2.loc[data2.index[(data2['Country Code']=='PHL')&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='PHL')&(data2['Year']==2017)][0],'Primary Energy Consumption']*1.028 #2018
data2.loc[data2.index[(data2['Country Code']=='PHL')&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='PHL')&(data2['Year']==2018)][0],'Primary Energy Consumption']*1.031 #2019
data2.loc[data2.index[(data2['Country Code']=='PHL')&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='PHL')&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.906 #2020

# Thailand
data2.loc[data2.index[(data2['Country Code']=='THA')&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='THA')&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.928 #2020

# Vietnam
data2.loc[data2.index[(data2['Country Code']=='VNM')&(data2['Year']==2014)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='VNM')&(data2['Year']==2013)][0],'Primary Energy Consumption']*1.118 #2015
data2.loc[data2.index[(data2['Country Code']=='VNM')&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='VNM')&(data2['Year']==2014)][0],'Primary Energy Consumption']*1.112 #2015
data2.loc[data2.index[(data2['Country Code']=='VNM')&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='VNM')&(data2['Year']==2015)][0],'Primary Energy Consumption']*1.079 #2016
data2.loc[data2.index[(data2['Country Code']=='VNM')&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='VNM')&(data2['Year']==2016)][0],'Primary Energy Consumption']*1.071 #2017
data2.loc[data2.index[(data2['Country Code']=='VNM')&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='VNM')&(data2['Year']==2017)][0],'Primary Energy Consumption']*1.127 #2018
data2.loc[data2.index[(data2['Country Code']=='VNM')&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='VNM')&(data2['Year']==2018)][0],'Primary Energy Consumption']*1.118 #2019
data2.loc[data2.index[(data2['Country Code']=='VNM')&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='VNM')&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.981 #2020

# Sri Lanka
data2.loc[data2.index[(data2['Country Code']=='LKA')&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='LKA')&(data2['Year']==2014)][0],'Primary Energy Consumption']*1.073 #2015
data2.loc[data2.index[(data2['Country Code']=='LKA')&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='LKA')&(data2['Year']==2015)][0],'Primary Energy Consumption']*1.065 #2016
data2.loc[data2.index[(data2['Country Code']=='LKA')&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='LKA')&(data2['Year']==2016)][0],'Primary Energy Consumption']*1.055 #2017
data2.loc[data2.index[(data2['Country Code']=='LKA')&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='LKA')&(data2['Year']==2017)][0],'Primary Energy Consumption']*1.059 #2018
data2.loc[data2.index[(data2['Country Code']=='LKA')&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='LKA')&(data2['Year']==2018)][0],'Primary Energy Consumption']*1.032 #2019
data2.loc[data2.index[(data2['Country Code']=='LKA')&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='LKA')&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.914 #2020

# Hong Kong
data2.loc[data2.index[(data2['Country Code']=='HKG')&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='HKG')&(data2['Year']==2014)][0],'Primary Energy Consumption']*1.028 #2015
data2.loc[data2.index[(data2['Country Code']=='HKG')&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='HKG')&(data2['Year']==2015)][0],'Primary Energy Consumption']*1.026 #2016
data2.loc[data2.index[(data2['Country Code']=='HKG')&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='HKG')&(data2['Year']==2016)][0],'Primary Energy Consumption']*1.072 #2017
data2.loc[data2.index[(data2['Country Code']=='HKG')&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='HKG')&(data2['Year']==2017)][0],'Primary Energy Consumption']*1.007 #2018
data2.loc[data2.index[(data2['Country Code']=='HKG')&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='HKG')&(data2['Year']==2018)][0],'Primary Energy Consumption']*0.953 #2019
data2.loc[data2.index[(data2['Country Code']=='HKG')&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='HKG')&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.749 #2020

# Cyprus
data2.loc[data2.index[(data2['Country Code']=='CYP')&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='CYP')&(data2['Year']==2014)][0],'Primary Energy Consumption']*1.031 #2015
data2.loc[data2.index[(data2['Country Code']=='CYP')&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='CYP')&(data2['Year']==2015)][0],'Primary Energy Consumption']*1.088 #2016
data2.loc[data2.index[(data2['Country Code']=='CYP')&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='CYP')&(data2['Year']==2016)][0],'Primary Energy Consumption']*1.027 #2017
data2.loc[data2.index[(data2['Country Code']=='CYP')&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='CYP')&(data2['Year']==2017)][0],'Primary Energy Consumption']*0.996 #2018
data2.loc[data2.index[(data2['Country Code']=='CYP')&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='CYP')&(data2['Year']==2018)][0],'Primary Energy Consumption']*1.001 #2019
data2.loc[data2.index[(data2['Country Code']=='CYP')&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='CYP')&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.871 #2020

# World
data2.loc[data2.index[(data2['Country Code']=='WLD')&(data2['Year']==1970)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='WLD')&(data2['Year']==1971)][0],'Primary Energy Consumption']*0.9608 #1970
data2.loc[data2.index[(data2['Country Code']=='WLD')&(data2['Year']==1969)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='WLD')&(data2['Year']==1970)][0],'Primary Energy Consumption']*0.9401 #1969
data2.loc[data2.index[(data2['Country Code']=='WLD')&(data2['Year']==1968)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='WLD')&(data2['Year']==1969)][0],'Primary Energy Consumption']*0.9372 #1968
data2.loc[data2.index[(data2['Country Code']=='WLD')&(data2['Year']==1967)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='WLD')&(data2['Year']==1968)][0],'Primary Energy Consumption']*0.9429 #1967
data2.loc[data2.index[(data2['Country Code']=='WLD')&(data2['Year']==1966)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='WLD')&(data2['Year']==1967)][0],'Primary Energy Consumption']*0.9638 #1966
data2.loc[data2.index[(data2['Country Code']=='WLD')&(data2['Year']==1965)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']=='WLD')&(data2['Year']==1966)][0],'Primary Energy Consumption']*0.9493 #1965
data2.loc[13297,'Primary Energy Consumption']=data2.loc[13296,'Primary Energy Consumption']*(1-0.045) #2020

In [37]:
Other_Southern_Africa=['AGO','BWA','MOZ','NAM','ZMB','ZWE']
Western_Africa=['BEN','CIV','CMR','GHA','NER','NGA','SEN','TGO']
Middle_Africa=['COG','COD', 'GAB']
Other_Europe=['ALB','BIH','MKD','MLT','MNE','SRB','XKX']  
Other_CIS=['ARM','GEO','KGZ','MDA','TJK']
Other_Middle_East=['BHR','JOR','LBN','SYR','YEM']
Other_South_America=['BOL','PRY','SUR','URY']
Other_Asia_Pacific=['BRN','KHM','MMR','MNG','NPL','PRK']
Central_America=['CRI','GTM','HND','NIC','PAN','SLV']
Other_Caribbean=['CUB','CUW','DOM','HTI','JAM']
Eastern_Africa=['ERI','ETH','KEN','MUS','TZA']
Other_Northern_Africa=['LBY','SDN','SSD','TUN']

# Other Southern Africa
for country in Other_Southern_Africa:
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2014)][0],'Primary Energy Consumption']*1.016 #2015
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2015)][0],'Primary Energy Consumption']*0.998 #2016
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2016)][0],'Primary Energy Consumption']*1.059 #2017
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2017)][0],'Primary Energy Consumption']*1.011 #2018
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2018)][0],'Primary Energy Consumption']*0.957 #2019
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.96 #2020
    
# Western Africa
for country in Western_Africa:
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2014)][0],'Primary Energy Consumption']*1.157 #2015
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2015)][0],'Primary Energy Consumption']*1.031 #2016
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2016)][0],'Primary Energy Consumption']*1.082 #2017
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2017)][0],'Primary Energy Consumption']*1.111 #2018
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2018)][0],'Primary Energy Consumption']*1.026 #2019
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.947 #2020
    
# Middle Africa
for country in Middle_Africa:
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2014)][0],'Primary Energy Consumption']*1.005 #2015
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2015)][0],'Primary Energy Consumption']*1.035 #2016
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2016)][0],'Primary Energy Consumption']*0.996 #2017
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2017)][0],'Primary Energy Consumption']*1.023 #2018
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2018)][0],'Primary Energy Consumption']*0.951 #2019
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.95 #2020 
    
# Easter Africa
for country in Eastern_Africa:
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2014)][0],'Primary Energy Consumption']*1.045 #2015
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2015)][0],'Primary Energy Consumption']*0.98 #2016
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2016)][0],'Primary Energy Consumption']*1.059 #2017
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2017)][0],'Primary Energy Consumption']*1.057 #2018
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2018)][0],'Primary Energy Consumption']*1.017 #2019
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.915 #2020     

# Other Northern Africa
for country in Other_Northern_Africa:
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2014)][0],'Primary Energy Consumption']*0.885 #2015
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2015)][0],'Primary Energy Consumption']*1.024 #2016
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2016)][0],'Primary Energy Consumption']*0.99 #2017
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2017)][0],'Primary Energy Consumption']*1.009 #2018
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2018)][0],'Primary Energy Consumption']*1.026 #2019
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.914 #2020     

# Other Europe
for country in Other_Europe:
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2014)][0],'Primary Energy Consumption']*1.066 #2015
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2015)][0],'Primary Energy Consumption']*1.061 #2016
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2016)][0],'Primary Energy Consumption']*0.995 #2017
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2017)][0],'Primary Energy Consumption']*1.046 #2018
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2018)][0],'Primary Energy Consumption']*1.033 #2019
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.895 #2020         

# Other CIS
for country in Other_CIS:
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2014)][0],'Primary Energy Consumption']*0.995 #2015
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2015)][0],'Primary Energy Consumption']*1.008 #2016
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2016)][0],'Primary Energy Consumption']*1.039 #2017
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2017)][0],'Primary Energy Consumption']*1.093 #2018
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2018)][0],'Primary Energy Consumption']*0.97 #2019
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2019)][0],'Primary Energy Consumption']*1.018 #2020         

# Other Middle East
for country in Other_Middle_East:
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2014)][0],'Primary Energy Consumption']*0.937 #2015
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2015)][0],'Primary Energy Consumption']*0.995 #2016
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2016)][0],'Primary Energy Consumption']*1.011 #2017
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2017)][0],'Primary Energy Consumption']*0.98 #2018
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2018)][0],'Primary Energy Consumption']*1.037 #2019
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.985 #2020         

# Other Asia Pacific
for country in Other_Asia_Pacific:
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2014)][0],'Primary Energy Consumption']*1.042 #2015
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2015)][0],'Primary Energy Consumption']*1.086 #2016
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2016)][0],'Primary Energy Consumption']*1.021 #2017
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2017)][0],'Primary Energy Consumption']*1.016 #2018
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2018)][0],'Primary Energy Consumption']*1.294 #2019
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2019)][0],'Primary Energy Consumption']*1.004 #2020         

# Other South America
for country in Other_South_America:
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2014)][0],'Primary Energy Consumption']*1.018 #2015
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2015)][0],'Primary Energy Consumption']*1.08 #2016
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2016)][0],'Primary Energy Consumption']*0.991 #2017
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2017)][0],'Primary Energy Consumption']*1.019 #2018
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2018)][0],'Primary Energy Consumption']*0.946 #2019
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.879 #2020        

# Central America
for country in Central_America:
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2014)][0],'Primary Energy Consumption']*1.09 #2015
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2015)][0],'Primary Energy Consumption']*1.042 #2016
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2016)][0],'Primary Energy Consumption']*1.029 #2017
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2017)][0],'Primary Energy Consumption']*1.015 #2018
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2018)][0],'Primary Energy Consumption']*1.056 #2019
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.906 #2020       

# Other Caribbean
for country in Other_Caribbean:
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2015)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2014)][0],'Primary Energy Consumption']*1.028 #2015
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2016)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2015)][0],'Primary Energy Consumption']*1.037 #2016
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2017)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2016)][0],'Primary Energy Consumption']*0.974 #2017
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2018)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2017)][0],'Primary Energy Consumption']*1.031 #2018
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2019)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2018)][0],'Primary Energy Consumption']*1.032 #2019
    data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2020)][0],'Primary Energy Consumption']=data2.loc[data2.index[(data2['Country Code']==country)&(data2['Year']==2019)][0],'Primary Energy Consumption']*0.934 #2020        

In [38]:
# Compile Primary Energy Consumption per capita based on Primary Energy Consumption figures from BP Statistical Review
data2['Primary Energy Consumption per capita'] = np.where(~data2['Primary Energy Consumption per capita'].isnull(),data2['Primary Energy Consumption per capita'],data2['Primary Energy Consumption']*23884.58966275*1000/data2['Population'])

# Derive the feature 'energy intensity', which equals to primary energy consumption divided by GDP at constant dollars
data2['Energy Intensity']=(data2['Primary Energy Consumption']*23884.58966275*1000)/data2['Constant GDP'] #kg of oil equivalent per $ constant GDP

In [39]:
# Supplement % share of renewable energy consumption using information from World Bank (also sourced from IEA)
renewable_wb = pd.read_sql('api_eg_fec_rnew_zs', db.config) # Renewable energy consumption (% of total final energy consumption)
renewable_wb.drop(renewable_wb.columns[[2,3]], axis=1, inplace=True)
col_year_list = list(renewable_wb.columns.values)
col_year_list = col_year_list[2:]
for i in col_year_list:
    renewable_wb[i] = renewable_wb[i].astype(float)
renewable_wb=pd.melt(renewable_wb, id_vars=['Country Name','Country Code'], value_vars=col_year_list) 
renewable_wb.rename(columns={'variable':'Year','value':'Renewable Energy Consumption Share WB'}, inplace=True)
renewable_wb['Year'] = renewable_wb['Year'].astype(int)
renewable_wb['Country Name'] = renewable_wb['Country Name'].str.strip() #strip leading blanks
data2=pd.merge(data2,renewable_wb,how='left',on=['Country Code','Country Name','Year'])
data2['Renewable Energy Consumption Share'] = np.where(~data2['Renewable Energy Consumption Share'].isnull(),data2['Renewable Energy Consumption Share'],data2['Renewable Energy Consumption Share WB'])
data2['Renewable Energy Consumption']=np.where(~data2['Renewable Energy Consumption'].isnull(),data2['Renewable Energy Consumption'],(data2['Renewable Energy Consumption Share']/100)*data2['Primary Energy Consumption'])
data2.drop(['Renewable Energy Consumption Share WB'],axis=1,inplace=True)

In [40]:
# Impute missing 2019 and 2020 renewable energy consumption share based on 2018 figures
data2_world_df = data2[data2['Country Code']=='WLD']
data2_temp=data2[data2['Country Code']!='WLD']
data2_temp=data2_temp.sort_values(by=['Country Code','Year']) 
codes=data2_temp['Country Code'].unique().tolist()
lst1=[]
lst2=[]
for code in codes:
    lst1.append(data2_temp[data2_temp['Country Code']==code]['Renewable Energy Consumption Share'].fillna(method='ffill'))
for item1 in lst1:
    for item2 in item1:
        lst2.append(item2)
data2_temp['Renewable Energy Consumption Share 2']=lst2
data2_temp.drop(['Renewable Energy Consumption Share'],axis=1,inplace=True)
data2_temp.rename(columns={'Renewable Energy Consumption Share 2':'Renewable Energy Consumption Share'}, inplace=True)
data2_temp['Renewable Energy Consumption']=np.where(~data2_temp['Renewable Energy Consumption'].isnull(),data2_temp['Renewable Energy Consumption'],data2_temp['Primary Energy Consumption']*data2_temp['Renewable Energy Consumption Share']/100)
data2_temp=data2_temp.sort_values(by=['Year','Country Code']) 
data2=data2_temp.append(data2_world_df,ignore_index=True)

### (D) Classifying countries into different regions and income groups

World Bank classify some 200 countries in the world into different regions or income group. The classification will also be adopted for data analysis purpose.

#### (D1) Region
* East Asia & Pacific
* Europe & Central Asia
* Latin America & Caribbean
* Middle East & North Africa
* North America
* South Asia
* Sub-Saharan Africa

#### (D2) Income group
* High income
* Upper middle income
* Lower middle income
* Low income

In [41]:
# Read country classification list from World Bank
classification_xls = pd.ExcelFile(path+'03 WorldBank_CLASS.xls')
classification = classification_xls.parse(0)
classification.drop(['World Bank list of economies (June 2020)','Unnamed: 1','Unnamed: 4',
                     'Unnamed: 7','Unnamed: 8','Unnamed: 2'],axis=1,inplace=True) 
classification.rename(columns={'Unnamed: 3':'Country Code',
                               'Unnamed: 5':'region','Unnamed: 6':'income_group'}, inplace=True)
classification.drop(classification.tail(8).index,inplace=True) 
classification.drop(classification.head(5).index,inplace=True) 
classification

# Merge the country classification with the merged dataset
data2 = pd.merge(data2,classification,how='left',on=['Country Code'])

In [42]:
data2

,Country Name,Country Code,Year,CO2 Emission,Population,Current GDP,Constant GDP,Manufacturing GDP,Medium to High Tech %,Export,Import,Real GDP Growth %,Urban Population Percent,Merchandise Export,Merchandise Import,Manufacturer Export Share,Manufacturer Export,CO2 Emission_Electricity,CO2 Emission_Building,CO2 Emission_Manufacturing,CO2 Emission_Other_Fuel,CO2 Emission_Fugitive,CO2 Emission_Transport,CO2 Emission_Energy_Subtotal,CO2 Emission_Bunkers,CO2 Emission_Industrial_Process,CO2 Emission per capita,Constant GDP per capita,Manufacturing %,Medium to High Tech GDP,CO2 Emission per constant GDP,Trade Openness,Share of Merchandise Export,Share of Merchandise Import,Industrial GDP,CO2 Country Share %,Manufacturing Country Share %,Primary Energy Consumption,Renewable Energy Consumption,Coal Consumption,Oil Consumption,Total Electricity Production,Electricity Production from Renewable,Primary Energy Consumption per capita,Fossil Energy Consumption Share,Renewable Electricity Production Share,Energy Intensity,Renewable Energy Consumption Share,region,income_group
0,Aruba,ABW,1960,619.0,5.420800e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.776000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.418979,NaN,1.622890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Latin America & Caribbean,High income
1,Afghanistan,AFG,1960,414.0,8.996967e+06,5.377778e+08,NaN,3.112143e+08,NaN,2.222223e+07,3.777778e+07,NaN,8.401000,5.000000e+07,8.700000e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.046016,NaN,57.870432,NaN,NaN,11.157027,9.297520,16.177685,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,South Asia,Low income
2,Angola,AGO,1960,549.0,5.454938e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.435000,1.240000e+08,1.280000e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.100643,NaN,9.174312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sub-Saharan Africa,Lower middle income
3,Albania,ALB,1960,2023.0,1.608800e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.705000,4.900000e+07,8.100000e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.257459,NaN,17.419305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Europe & Central Asia,Upper middle income
4,Andorra,AND,1960,NaN,1.341000e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.450000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.658603,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Europe & Central Asia,High income
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13391,World,WLD,2016,35452459.0,7.424485e+09,7.620000e+13,7.700000e+13,1.212278e+13,36.925752,2.270765e+13,2.272194e+13,7.713090e+13,54.370833,1.612320e+13,1.626320e+13,67.768793,1.092650e+13,17010000.0,3259670.0,6625090.0,669330.0,227670.0,7740720.0,32924060.0,1284090.0,1501450.0,4.775073,10371.090074,15.909159,4.476427e+12,0.460422,59.618888,21.159055,21.342782,2.004590e+13,6.308267e+06,3.558232e+06,572826.9750,77475.13790,156117.9159,188744.1009,24947744.76,5891550.903,1842.786095,60.203523,23.615565,0.177685,13.525051,NaN,NaN
13392,World,WLD,2017,35925738.0,7.509410e+09,8.110000e+13,7.960000e+13,1.271146e+13,31.564586,2.252503e+13,2.197962e+13,7.974330e+13,54.823831,1.782960e+13,1.804390e+13,69.575431,1.240502e+13,15026540.0,2801770.0,6149360.0,621430.0,225800.0,6157420.0,28235490.0,1194730.0,1472970.0,4.784096,10600.033502,15.673804,4.012318e+12,0.451328,54.876254,21.984710,22.248952,2.168380e+13,8.328675e+04,3.445306e+05,585039.1235,80172.82775,158759.3456,191672.3894,25688502.59,6250190.754,1860.787862,59.898855,24.330693,0.175545,13.703840,NaN,NaN
13393,World,WLD,2018,36646140.0,7.592476e+09,8.610000e+13,8.210000e+13,1.345212e+13,31.042139,2.470781e+13,2.415435e+13,8.232840e+13,55.271710,1.964610e+13,1.990070e+13,69.253146,1.360554e+13,15384600.0,2885780.0,6120700.0,624070.0,20244

In [43]:
#Save data2 as csv file
print(data2.columns)
print(data2.shape)

if write_to_db:
    data2.to_sql("consolidated_data", db.config,index = False, if_exists = 'replace')
else:
    data2.to_csv("consolidated_data", index = False)

print('Upload to Database Completed.')

Index(['Country Name', 'Country Code', 'Year', 'CO2 Emission', 'Population',
       'Current GDP', 'Constant GDP', 'Manufacturing GDP',
       'Medium to High Tech %', 'Export', 'Import', 'Real GDP Growth %',
       'Urban Population Percent', 'Merchandise Export', 'Merchandise Import',
       'Manufacturer Export Share', 'Manufacturer Export',
       'CO2 Emission_Electricity', 'CO2 Emission_Building',
       'CO2 Emission_Manufacturing', 'CO2 Emission_Other_Fuel',
       'CO2 Emission_Fugitive', 'CO2 Emission_Transport',
       'CO2 Emission_Energy_Subtotal', 'CO2 Emission_Bunkers',
       'CO2 Emission_Industrial_Process', 'CO2 Emission per capita',
       'Constant GDP per capita', 'Manufacturing %', 'Medium to High Tech GDP',
       'CO2 Emission per constant GDP', 'Trade Openness',
       'Share of Merchandise Export', 'Share of Merchandise Import',
       'Industrial GDP', 'CO2 Country Share %',
       'Manufacturing Country Share %', 'Primary Energy Consumption',
       'Renewa

In [44]:

energy_balance.columns

Index(['Country', 'Product',    'Flow',      1971,      1972,      1973,
            1974,      1975,      1976,      1977,      1978,      1979,
            1980,      1981,      1982,      1983,      1984,      1985,
            1986,      1987,      1988,      1989,      1990,      1991,
            1992,      1993,      1994,      1995,      1996,      1997,
            1998,      1999,      2000,      2001,      2002,      2003,
            2004,      2005,      2006,      2007,      2008,      2009,
            2010,      2011,      2012,      2013,      2014,      2015,
            2016,      2017,      2018,      2019,      2020],
      dtype='object')